In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
from gym import spaces
import numpy as np
import six

# Some notes:
#
#   These represent properties of the documents. Presumably we can add more or
#   remove documents from the candidate set. But these do not include features
#   that depend on both the user and the document.
#


class CandidateSet(object):
  """Class to represent a collection of AbstractDocuments.

     The candidate set is represented as a hashmap (dictionary), with documents
     indexed by their document ID.
  """

  def __init__(self):
    """Initializes a document candidate set with 0 documents."""
    self._documents = {}

  def size(self):
    """Returns an integer, the number of documents in this candidate set."""
    return len(self._documents)

  def get_all_documents(self):
    """Returns all documents."""
    return self.get_documents(self._documents.keys())

  def get_documents(self, document_ids):
    """Gets the documents associated with the specified document IDs.

    Args:
      document_ids: an array representing indices into the candidate set.
        Indices can be integers or string-encoded integers.

    Returns:
      (documents) an ordered list of AbstractDocuments associated with the
        document ids.
    """
    return [self._documents[int(k)] for k in document_ids]

  def add_document(self, document):
    """Adds a document to the candidate set."""
    self._documents[document.doc_id()] = document

  def remove_document(self, document):
    """Removes a document from the set (to simulate a changing corpus)."""
    del self._documents[document.doc_id()]

  def create_observation(self):
    """Returns a dictionary of observable features of documents."""
    return {
        str(k): self._documents[k].create_observation()
        for k in self._documents.keys()
    }

  def observation_space(self):
    return spaces.Dict({
        str(k): self._documents[k].observation_space()
        for k in self._documents.keys()
    })


@six.add_metaclass(abc.ABCMeta)
class AbstractDocumentSampler(object):
  """Abstract class to sample documents."""

  def __init__(self, doc_ctor, seed=0):
    self._doc_ctor = doc_ctor
    self._seed = seed
    self.reset_sampler()

  def reset_sampler(self):
    self._rng = np.random.RandomState(self._seed)

  @abc.abstractmethod
  def sample_document(self):
    """Samples and return an instantiation of AbstractDocument."""

  def get_doc_ctor(self):
    """Returns the constructor/class of the documents that will be sampled."""
    return self._doc_ctor

  @property
  def num_clusters(self):
    """Returns the number of document clusters. Returns 0 if not applicable."""
    return 0

  def update_state(self, documents, responses):
    """Update document state (if needed) given user's (or users') responses."""
    pass


@six.add_metaclass(abc.ABCMeta)
class AbstractDocument(object):
  """Abstract class to represent a document and its properties."""

  # Number of features to represent the document.
  NUM_FEATURES = None

  def __init__(self, doc_id):
    self._doc_id = doc_id  # Unique identifier for the document

  def doc_id(self):
    """Returns the document ID."""
    return self._doc_id

  @abc.abstractmethod
  def create_observation(self):
    """Returns observable properties of this document as a float array."""

  @classmethod
  @abc.abstractmethod
  def observation_space(cls):
    """Gym space that defines how documents are represented."""

In [ ]:
"""Abstract classes that encode a user's state and dynamics."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
from gym import spaces
import numpy as np
import six


@six.add_metaclass(abc.ABCMeta)
class AbstractResponse(object):
  """Abstract class to model a user response."""

  @staticmethod
  @abc.abstractmethod
  def response_space():
    """ArraySpec that defines how a single response is represented."""

  @abc.abstractmethod
  def create_observation(self):
    """Creates a tensor observation of this response."""


@six.add_metaclass(abc.ABCMeta)
class AbstractUserState(object):
  """Abstract class to represent a user's state."""

  # Number of features to represent the user's interests.
  NUM_FEATURES = None

  @abc.abstractmethod
  def create_observation(self):
    """Generates obs of underlying state to simulate partial observability.

    Returns:
      obs: A float array of the observed user features.
    """

  @staticmethod
  @abc.abstractmethod
  def observation_space():
    """Gym.spaces object that defines how user states are represented."""


@six.add_metaclass(abc.ABCMeta)
class AbstractUserSampler(object):
  """Abstract class to sample users."""

  def __init__(self, user_ctor, seed=0):
    """Creates a new user state sampler.

    User states of the type user_ctor are sampled.

    Args:
      user_ctor: A class/constructor for the type of user states that will be
        sampled.
      seed: An integer for a random seed.
    """
    self._user_ctor = user_ctor
    self._seed = seed
    self.reset_sampler()

  def reset_sampler(self):
    self._rng = np.random.RandomState(self._seed)

  @abc.abstractmethod
  def sample_user(self):
    """Creates a new instantiation of this user's hidden state parameters."""

  def get_user_ctor(self):
    """Returns the constructor/class of the user states that will be sampled."""
    return self._user_ctor


@six.add_metaclass(abc.ABCMeta)
class AbstractUserModel(object):
  """Abstract class to represent an encoding of a user's dynamics."""

  def __init__(self, response_model_ctor, user_sampler, slate_size):
    """Initializes a new user model.

    Args:
      response_model_ctor: A class/constructor representing the type of
        responses this model will generate.
      user_sampler: An instance of AbstractUserSampler that can generate
        initial user states from an inital state distribution.
      slate_size: integer number of documents that can be served to the user at
        any interaction.
    """
    if not response_model_ctor:
      raise TypeError('response_model_ctor is a required callable')

    self._user_sampler = user_sampler
    self._user_state = self._user_sampler.sample_user()
    self._response_model_ctor = response_model_ctor
    self._slate_size = slate_size

  ## Transition model
  @abc.abstractmethod
  def update_state(self, slate_documents, responses):
    """Updates the user's state based on the slate and document selected.

    Args:
      slate_documents: A list of AbstractDocuments for items in the slate.
      responses: A list of AbstractResponses for each item in the slate.
    Updates: The user's hidden state.
    """

  def reset(self):
    """Resets the user."""
    self._user_state = self._user_sampler.sample_user()

  def reset_sampler(self):
    """Resets the sampler."""
    self._user_sampler.reset_sampler()

  @abc.abstractmethod
  def is_terminal(self):
    """Returns a boolean indicating whether this session is over."""

  ## Choice model
  @abc.abstractmethod
  def simulate_response(self, documents):
    """Simulates the user's response to a slate of documents.

    This could involve simulating models of attention, as well as random
    sampling for selection from scored documents.

    Args:
      documents: a list of AbstractDocuments

    Returns:
      (response) a list of AbstractResponse objects for each slate item
    """

  def response_space(self):
    res_space = self._response_model_ctor.response_space()
    return spaces.Tuple(tuple([
        res_space,
    ] * self._slate_size))

  def get_response_model_ctor(self):
    """Returns a constructor for the type of response this model will create."""
    return self._response_model_ctor

  def observation_space(self):
    """A Gym.spaces object that describes possible user observations."""
    return self._user_state.observation_space()

  def create_observation(self):
    """Emits obesrvation about user's state."""
    return self._user_state.create_observation()

In [ ]:
"""Abstract classes that encode a user's state and dynamics."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import numpy as np
import six


def softmax(vector):
  """Computes the softmax of a vector."""
  normalized_vector = np.array(vector) - np.max(
      vector)  # For numerical stability
  return np.exp(normalized_vector) / np.sum(np.exp(normalized_vector))


@six.add_metaclass(abc.ABCMeta)
class AbstractChoiceModel(object):
  """Abstract class to represent the user choice model.

  Each user has a choice model.
  """
  _scores = None
  _score_no_click = None

  @abc.abstractmethod
  def score_documents(self, user_state, doc_obs):
    """Computes unnormalized scores of documents in the slate given user state.

    Args:
      user_state: An instance of AbstractUserState.
      doc_obs: A numpy array that represents the observation of all documents in
        the slate.
    Attributes:
      scores: A numpy array that stores the scores of all documents.
      score_no_click: A float that represents the score for the action of
        picking no document.
    """

  @property
  def scores(self):
    return self._scores

  @property
  def score_no_click(self):
    return self._score_no_click

  @abc.abstractmethod
  def choose_item(self):
    """Returns selected index of document in the slate.

    Returns:
      selected_index: a integer indicating which item was chosen, or None if
        none were selected.
    """


class NormalizableChoiceModel(AbstractChoiceModel):
  """A normalizable choice model."""

  @staticmethod
  def _score_documents_helper(user_state, doc_obs):
    scores = np.array([])
    for doc in doc_obs:
      scores = np.append(scores, user_state.score_document(doc))
    return scores

  def choose_item(self):
    all_scores = np.append(self._scores, self._score_no_click)
    all_probs = all_scores / np.sum(all_scores)
    selected_index = np.random.choice(len(all_probs), p=all_probs)
    if selected_index == len(all_probs) - 1:
      selected_index = None
    return selected_index


class MultinomialLogitChoiceModel(NormalizableChoiceModel):
  """A multinomial logit choice model.

   Samples item x in scores according to
     p(x) = exp(x) / Sum_{y in scores} exp(y)

   Args:
     choice_features: a dict that stores the features used in choice model:
       `no_click_mass`: a float indicating the mass given to a no click option.
  """

  def __init__(self, choice_features):
    self._no_click_mass = choice_features.get('no_click_mass', -float('Inf'))

  def score_documents(self, user_state, doc_obs):
    logits = self._score_documents_helper(user_state, doc_obs)
    logits = np.append(logits, self._no_click_mass)
    # Use softmax scores instead of exponential scores to avoid overflow.
    all_scores = softmax(logits)
    self._scores = all_scores[:-1]
    self._score_no_click = all_scores[-1]


class MultinomialProportionalChoiceModel(NormalizableChoiceModel):
  """A multinomial proportional choice function.

  Samples item x in scores according to
    p(x) = x - min_normalizer / sum(x - min_normalizer)

  Attributes:
    min_normalizer: A float (<= 0) used to offset the scores to be positive.
      Specifically, if the scores have negative elements, then they do not
      form a valid probability distribution for sampling. Subtracting the
      least expected element is one heuristic for normalization.
    no_click_mass: An optional float indicating the mass given to a no click
      option
  """

  def __init__(self, choice_features):
    self._min_normalizer = choice_features.get('min_normalizer')
    self._no_click_mass = choice_features.get('no_click_mass', 0)

  def score_documents(self, user_state, doc_obs):
    scores = self._score_documents_helper(user_state, doc_obs)
    all_scores = np.append(scores, self._no_click_mass)
    all_scores = all_scores - self._min_normalizer
    assert all_scores[
        all_scores <
        0.0].size == 0, 'Normalized scores have non-positive elements.'
    self._scores = all_scores[:-1]
    self._score_no_click = all_scores[-1]


class CascadeChoiceModel(NormalizableChoiceModel):
  """The base class for cascade choice models.

  Attributes:
    attention_prob: The probability of examining a document i given document i -
      1 not clicked.
    score_scaling: A multiplicative factor to convert score of document i to the
      click probability of examined document i.

  Raises:
    ValueError: if either attention_prob or base_attention_prob is invalid.
  """

  def __init__(self, choice_features):
    self._attention_prob = choice_features.get('attention_prob', 1.0)
    self._score_scaling = choice_features.get('score_scaling')
    if self._attention_prob < 0.0 or self._attention_prob > 1.0:
      raise ValueError('attention_prob must be in [0,1].')
    if self._score_scaling < 0.0:
      raise ValueError('score_scaling must be positive.')

  def _positional_normalization(self, scores):
    """Computes the click probability of each document in _scores.

    The probability to click item i conditioned on unclicked item i - 1 is:
      attention_prob * score_scaling * score(i)
    We also compute the probability of not clicking any items in _score_no_click
    Because they are already probabilities, the normlaization in choose_item
    is no-op but we utilize random choice there.

    Args:
      scores: normalizable scores.
    """
    self._score_no_click = 1.0
    for i in range(len(scores)):
      s = self._score_scaling * scores[i]
      assert s <= 1.0, ('score_scaling cannot convert score %f into a '
                        'probability') % scores[i]
      scores[i] = self._score_no_click * self._attention_prob * s
      self._score_no_click *= (1.0 - self._attention_prob * s)
    self._scores = scores


class ExponentialCascadeChoiceModel(CascadeChoiceModel):
  """An exponential cascade choice model.

  Clicks the item at position i according to
    p(i) = attention_prob * score_scaling * exp(score(i))
  by going through the slate in order, and stopping once an item has been
  clicked.
  """

  def score_documents(self, user_state, doc_obs):
    scores = self._score_documents_helper(user_state, doc_obs)
    scores = np.exp(scores)
    self._positional_normalization(scores)


class ProportionalCascadeChoiceModel(CascadeChoiceModel):
  """A proportional cascade choice model.

  Clicks the item at position i according to
    attention_prob * score_scaling * (score(i) - min_normalizer)
  by going through the slate in order, and stopping once an item has been
  clicked.
  """

  def __init__(self, choice_features):
    self._min_normalizer = choice_features.get('min_normalizer')
    super(ProportionalCascadeChoiceModel, self).__init__(choice_features)

  def score_documents(self, user_state, doc_obs):
    scores = self._score_documents_helper(user_state, doc_obs)
    scores = scores - self._min_normalizer
    assert not scores[
        scores < 0.0], 'Normalized scores have non-positive elements.'
    self._positional_normalization(scores)

In [ ]:
"""Class to represent the environment in the recommender system setting.

   Thus, it models things such as (1) the user's state, for example his/her
   interests and circumstances, (2) the documents available to suggest from and
   their properties, (3) simulates the selection of an item in the slate (or a
   no-op/quit), and (4) models the change in a user's state based on the slate
   presented and the document selected.

   The agent interacting with the environment is the recommender system.  The
   agent receives the state, which is an observation of the user's state and
   observations of the candidate documents. The agent then provides an action,
   which is a slate (an array of indices into the candidate set).

   The goal of the agent is to learn a recommendation policy: a policy that
   serves the user a slate (action) based on user and document features (state)
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import collections
import itertools

import six


@six.add_metaclass(abc.ABCMeta)
class AbstractEnvironment(object):
  """Abstract class representing the recommender system environment.

  Attributes:
    user_model: An list or single instantiation of AbstractUserModel
      representing the user/users.
    document_sampler: An instantiation of AbstractDocumentSampler.
    num_candidates: An integer representing the size of the candidate_set.
    slate_size: An integer representing the slate size.
    candidate_set: An instantiation of CandidateSet.
    num_clusters: An integer representing the number of document clusters.
  """

  def __init__(self,
               user_model,
               document_sampler,
               num_candidates,
               slate_size,
               resample_documents=True):
    """Initializes a new simulation environment.

    Args:
      user_model: An instantiation of AbstractUserModel or list of such
        instantiations
      document_sampler: An instantiation of AbstractDocumentSampler
      num_candidates: An integer representing the size of the candidate_set
      slate_size: An integer representing the slate size
      resample_documents: A boolean indicating whether to resample the candidate
        set every step
    """
    self._user_model = user_model
    self._document_sampler = document_sampler
    self._slate_size = slate_size
    self._num_candidates = num_candidates
    self._resample_documents = resample_documents

    # Create a candidate set.
    self._do_resample_documents()
    assert (slate_size <= num_candidates
           ), 'Slate size %d cannot be larger than number of candidates %d' % (
               slate_size, num_candidates)

  def _do_resample_documents(self):
    # TODO(sanmit): eventually model this creation with content creators.
    self._candidate_set = CandidateSet()
    for _ in range(self._num_candidates):
      self._candidate_set.add_document(self._document_sampler.sample_document())

  @abc.abstractmethod
  def reset(self):
    """Resets the environment and return the first observation.

    Returns:
      user_obs: An array of floats representing observations of the user's
        current state
      doc_obs: An OrderedDict of document observations keyed by document ids
    """

  @abc.abstractmethod
  def reset_sampler(self):
    """Resets the relevant samplers of documents and user/users."""

  @property
  def num_candidates(self):
    return self._num_candidates

  @property
  def slate_size(self):
    return self._slate_size

  @property
  def candidate_set(self):
    return self._candidate_set

  @property
  def user_model(self):
    return self._user_model

  @abc.abstractmethod
  def step(self, slate):
    """Executes the action, returns next state observation and reward.

    Args:
      slate: An integer array of size slate_size (or list of such arrays), where
      each element is an index into the set of current_documents presented.

    Returns:
      user_obs: A gym observation representing the user's next state
      doc_obs: A list of observations of the documents
      responses: A list of AbstractResponse objects for each item in the slate
      done: A boolean indicating whether the episode has terminated
    """


class SingleUserEnvironment(AbstractEnvironment):
  """Class to represent the environment with one user.

  Attributes:
    user_model: An instantiation of AbstractUserModel that represents a user.
    document_sampler: An instantiation of AbstractDocumentSampler.
    num_candidates: An integer representing the size of the candidate_set.
    slate_size: An integer representing the slate size.
    candidate_set: An instantiation of CandidateSet.
    num_clusters: An integer representing the number of document clusters.
  """

  def reset(self):
    """Resets the environment and return the first observation.

    Returns:
      user_obs: An array of floats representing observations of the user's
        current state
      doc_obs: An OrderedDict of document observations keyed by document ids
    """
    self._user_model.reset()
    user_obs = self._user_model.create_observation()
    if self._resample_documents:
      self._do_resample_documents()
    self._current_documents = collections.OrderedDict(
        self._candidate_set.create_observation())
    return (user_obs, self._current_documents)

  def reset_sampler(self):
    """Resets the relevant samplers of documents and user/users."""
    self._document_sampler.reset_sampler()
    self._user_model.reset_sampler()

  def step(self, slate):
    """Executes the action, returns next state observation and reward.

    Args:
      slate: An integer array of size slate_size, where each element is an index
        into the set of current_documents presented

    Returns:
      user_obs: A gym observation representing the user's next state
      doc_obs: A list of observations of the documents
      responses: A list of AbstractResponse objects for each item in the slate
      done: A boolean indicating whether the episode has terminated
    """

    assert (len(slate) <= self._slate_size
           ), 'Received unexpectedly large slate size: expecting %s, got %s' % (
               self._slate_size, len(slate))

    # Get the documents associated with the slate
    doc_ids = list(self._current_documents)  # pytype: disable=attribute-error

    mapped_slate = [doc_ids[x] for x in slate]

    documents = self._candidate_set.get_documents(mapped_slate)

    # Simulate the user's response
    responses = self._user_model.simulate_response(documents)

    # Update the user's state.
    self._user_model.update_state(documents, responses)

    # Update the documents' state.
    self._document_sampler.update_state(documents, responses)

    # Obtain next user state observation.
    user_obs = self._user_model.create_observation()

    # Check if reaches a terminal state and return.
    done = self._user_model.is_terminal()

    # Optionally, recreate the candidate set to simulate candidate
    # generators for the next query.
    if self._resample_documents:
      self._do_resample_documents()

    # Create observation of candidate set.
    self._current_documents = collections.OrderedDict(
        self._candidate_set.create_observation())

    return (user_obs, self._current_documents, responses, done)


Environment = SingleUserEnvironment  # for backwards compatability


class MultiUserEnvironment(AbstractEnvironment):
  """Class to represent environment with multiple users.

  Attributes:
    user_model: A list of AbstractUserModel instances that represent users.
    num_users: An integer representing the number of users.
    document_sampler: An instantiation of AbstractDocumentSampler.
    num_candidates: An integer representing the size of the candidate_set.
    slate_size: An integer representing the slate size.
    candidate_set: An instantiation of CandidateSet.
    num_clusters: An integer representing the number of document clusters.
  """

  def reset(self):
    """Resets the environment and return the first observation.

    Returns:
      user_obs: An array of floats representing observations of the user's
        current state
      doc_obs: An OrderedDict of document observations keyed by document ids
    """
    for user_model in self.user_model:
      user_model.reset()
    user_obs = [
        user_model.create_observation() for user_model in self.user_model
    ]
    if self._resample_documents:
      self._do_resample_documents()
    self._current_documents = collections.OrderedDict(
        self._candidate_set.create_observation())
    return (user_obs, self._current_documents)

  def reset_sampler(self):
    self._document_sampler.reset_sampler()
    for user_model in self.user_model:
      user_model.reset_sampler()

  @property
  def num_users(self):
    return len(self.user_model)

  def step(self, slates):
    """Executes the action, returns next state observation and reward.

    Args:
      slates: A list of slates, where each slate is an integer array of size
        slate_size, where each element is an index into the set of
        current_documents presented

    Returns:
      user_obs: A list of gym observation representing all users' next state
      doc_obs: A list of observations of the documents
      responses: A list of AbstractResponse objects for each item in the slate
      done: A boolean indicating whether the episode has terminated
    """

    assert (len(slates) == self.num_users
           ), 'Received unexpected number of slates: expecting %s, got %s' % (
               self._slate_size, len(slates))
    for i, slate in enumerate(slates):
      assert (len(slate) <= self._slate_size
             ), 'Slate %s is too large : expecting size %s, got %s' % (
                 i, self._slate_size, len(slate))

    all_user_obs = []
    all_documents = []  # Accumulate documents served to each user.
    all_responses = []  # Accumulate each user's responses to served documents.
    for user_model, slate in zip(self.user_model, slates):
      # Get the documents associated with the slate
      doc_ids = list(self._current_documents)  # pytype: disable=attribute-error
      mapped_slate = [doc_ids[x] for x in slate]
      documents = self._candidate_set.get_documents(mapped_slate)
      if user_model.is_terminal():
        responses = []
      else:
        # Simulate the user's response
        responses = user_model.simulate_response(documents)

        # Update the user's state.
        user_model.update_state(documents, responses)

      # Obtain next user state observation.
      all_user_obs.append(user_model.create_observation())
      all_documents.append(documents)
      all_responses.append(responses)

    def flatten(list_):
      return list(itertools.chain(*list_))

    # Update the documents' state.
    self._document_sampler.update_state(
        flatten(all_documents), flatten(all_responses))

    # Check if reaches a terminal state and return.
    done = all([user_model.is_terminal() for user_model in self.user_model])

    # Optionally, recreate the candidate set to simulate candidate
    # generators for the next query.
    if self._resample_documents:
      self._do_resample_documents()

    # Create observation of candidate set.
    self._current_documents = collections.OrderedDict(
        self._candidate_set.create_observation())

    return (all_user_obs, self._current_documents, all_responses, done)

In [ ]:
"""A wrapper for using Gym environment."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import gym
from gym import spaces
import numpy as np


def _dummy_metrics_aggregator(responses, metrics, info):
  del responses  # Unused.
  del metrics  # Unused.
  del info  # Unused.
  return


def _dummy_metrics_writer(metrics, add_summary_fn):
  del metrics  # Unused.
  del add_summary_fn  # Unused.
  return


class RecSimGymEnv(gym.Env):
  """Class to wrap recommender system environment to gym.Env.

  Attributes:
    game_over: A boolean indicating whether the current game has finished
    action_space: A gym.spaces object that specifies the space for possible
      actions.
    observation_space: A gym.spaces object that specifies the space for possible
      observations.
  """

  def __init__(self,
               raw_environment,
               reward_aggregator,
               metrics_aggregator=_dummy_metrics_aggregator,
               metrics_writer=_dummy_metrics_writer):
    """Initializes a RecSim environment conforming to gym.Env.

    Args:
      raw_environment: A recsim recommender system environment.
      reward_aggregator: A function mapping a list of responses to a number.
      metrics_aggregator: A function aggregating metrics over all steps given
        responses and response_names.
      metrics_writer:  A function writing final metrics to TensorBoard.
    """
    self._environment = raw_environment
    self._reward_aggregator = reward_aggregator
    self._metrics_aggregator = metrics_aggregator
    self._metrics_writer = metrics_writer
    self.reset_metrics()

  @property
  def environment(self):
    """Returns the recsim recommender system environment."""
    return self._environment

  @property
  def game_over(self):
    return False

  @property
  def action_space(self):
    """Returns the action space of the environment.

    Each action is a vector that specified document slate. Each element in the
    vector corresponds to the index of the document in the candidate set.
    """
    action_space = spaces.MultiDiscrete(
        self._environment.num_candidates * np.ones(
            (self._environment.slate_size,)
        ))
    if isinstance(self._environment, MultiUserEnvironment):
      action_space = spaces.Tuple([action_space] * self._environment.num_users)
    return action_space

  @property
  def observation_space(self):
    """Returns the observation space of the environment.

    Each observation is a dictionary with three keys `user`, `doc` and
    `response` that includes observation about user state, document and user
    response, respectively.
    """
    if isinstance(self._environment, MultiUserEnvironment):
      user_obs_space = self._environment.user_model[0].observation_space()
      resp_obs_space = self._environment.user_model[0].response_space()
      user_obs_space = spaces.Tuple(
          [user_obs_space] * self._environment.num_users)
      resp_obs_space = spaces.Tuple(
          [resp_obs_space] * self._environment.num_users)

    if isinstance(self._environment, SingleUserEnvironment):
      user_obs_space = self._environment.user_model.observation_space()
      resp_obs_space = self._environment.user_model.response_space()

    return spaces.Dict({
        'user': user_obs_space,
        'doc': self._environment.candidate_set.observation_space(),
        'response': resp_obs_space,
    })

  def step(self, action):
    """Runs one timestep of the environment's dynamics.

    When end of episode is reached, you are responsible for calling `reset()`
    to reset this environment's state. Accepts an action and returns a tuple
    (observation, reward, done, info).

    Args:
      action (object): An action provided by the environment

    Returns:
      A four-tuple of (observation, reward, done, info) where:
        observation (object): agent's observation that include
          1. User's state features
          2. Document's observation
          3. Observation about user's slate responses.
        reward (float) : The amount of reward returned after previous action
        done (boolean): Whether the episode has ended, in which case further
          step() calls will return undefined results
        info (dict): Contains responses for the full slate for
          debugging/learning.
    """
    user_obs, doc_obs, responses, done = self._environment.step(action)
    # print("RecSim responses: ", responses)
    if isinstance(self._environment, MultiUserEnvironment):
      all_responses = tuple(
          tuple(
              response.create_observation() for response in single_user_resps
              ) for single_user_resps in responses
          )
    else:  # single user environment
      if isinstance(responses, list):  # Check if responses is a list or iterable
          all_responses = tuple(
             response.create_observation() for response in responses)
      else:
        all_responses = (responses,)  # If it's not a list, make it a single-element tuple
    obs = dict(
        user=user_obs,
        doc=doc_obs,
        response=all_responses)

    # extract rewards from responses
    reward = self._reward_aggregator(responses)
    info = self.extract_env_info()
    return obs, reward, done, info

  def reset(self):
    user_obs, doc_obs = self._environment.reset()
    return dict(user=user_obs, doc=doc_obs, response=None)

  def reset_sampler(self):
    self._environment.reset_sampler()

  def render(self, mode='human'):
    raise NotImplementedError

  def close(self):
    raise NotImplementedError

  def seed(self, seed=None):
    np.random.seed(seed=seed)

  def extract_env_info(self):
    info = {'env': self._environment}
    return info

  def reset_metrics(self):
    """Resets every metric to zero.

    We reset metrics for every iteration but not every episode. On the other
    hand, reset() gets called for every episode.
    """
    self._metrics = collections.defaultdict(float)

  def update_metrics(self, responses, info=None):
    """Updates metrics with one step responses."""
    self._metrics = self._metrics_aggregator(
        responses, self._metrics, info)

  def write_metrics(self, add_summary_fn):
    """Writes metrics to TensorBoard by calling add_summary_fn."""
    self._metrics_writer(self._metrics, add_summary_fn)

In [ ]:
r"""An executable class to run agents in the simulator."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time

from absl import flags
from dopamine.discrete_domains import checkpointer
import gin
from gym import spaces
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()


flags.DEFINE_bool(
    'debug_mode', False,
    'If set to true, the agent will output in-episode statistics '
    'to Tensorboard. Disabled by default as this results in '
    'slower training.')
flags.DEFINE_string('agent_name', None, 'Name of the agent.')
flags.DEFINE_string('base_dir', None,
                    'Base directory to host all required sub-directories.')
flags.DEFINE_string(
    'environment_name', 'interest_evolution',
    'The environment with which to run the experiment. Supported choices are '
    '{interest_evolution, interest_exploration}.')
flags.DEFINE_string(
    'episode_log_file', '',
    'Filename under base_dir to output simulated episodes in SequenceExample.')
flags.DEFINE_multi_string(
    'gin_files', [], 'List of paths to gin configuration files (e.g.'
    '"third_party/py/dopamine/agents/dqn/dqn.gin").')
flags.DEFINE_multi_string(
    'gin_bindings', [],
    'Gin bindings to override the values set in the config files '
    '(e.g. "runner_lib.Runner.max_steps_per_episode=100')


FLAGS = flags.FLAGS


def load_gin_configs(gin_files, gin_bindings):
  """Loads gin configuration files.

  Args:
    gin_files: list, of paths to the gin configuration files for this
      experiment.
    gin_bindings: list, of gin parameter bindings to override the values in the
      config files.
  """
  gin.parse_config_files_and_bindings(
      gin_files, bindings=gin_bindings, skip_unknown=False)


@gin.configurable
class Runner(object):
  """Object that handles running experiments.

  Here we use the term 'experiment' to mean simulating interactions between the
  agent and the environment and reporting some statistics pertaining to these
  interactions.
  """

  _output_dir = None
  _checkpoint_dir = None
  _agent = None
  _checkpointer = None

  def __init__(self,
               base_dir,
               create_agent_fn,
               env,
               episode_log_file='',
               checkpoint_file_prefix='ckpt',
               max_steps_per_episode=27000):
    """Initializes the Runner object in charge of running a full experiment.

    Args:
      base_dir: str, the base directory to host all required sub-directories.
      create_agent_fn: A function that takes as args a Tensorflow session and an
        environment, and returns an agent.
      env: A Gym environment for running the experiments.
      episode_log_file: Path to output simulated episodes in tf.SequenceExample.
        Disable logging if episode_log_file is an empty string.
      checkpoint_file_prefix: str, the prefix to use for checkpoint files.
      max_steps_per_episode: int, maximum number of steps after which an episode
        terminates.
    """
    tf.logging.info('max_steps_per_episode = %s', max_steps_per_episode)

    if base_dir is None:
      raise ValueError('Missing base_dir.')

    self._base_dir = base_dir
    self._create_agent_fn = create_agent_fn
    self._env = env
    self._checkpoint_file_prefix = checkpoint_file_prefix
    self._max_steps_per_episode = max_steps_per_episode
    self._episode_log_file = episode_log_file
    self._episode_writer = None

  def _set_up(self, eval_mode):
    """Sets up the runner by creating and initializing the agent."""
    # Reset the tf default graph to avoid name collisions from previous runs
    # before doing anything else.
    tf.reset_default_graph()
    self._summary_writer = tf.summary.FileWriter(self._output_dir)
    if self._episode_log_file:
      self._episode_writer = tf.io.TFRecordWriter(
          os.path.join(self._output_dir, self._episode_log_file))
    # Set up a session and initialize variables.
    self._sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    self._agent = self._create_agent_fn(
        self._sess,
        self._env,
        summary_writer=self._summary_writer,
        eval_mode=eval_mode)
    # type check: env/agent must both be multi- or single-user
    if self._agent.multi_user and not isinstance(
        self._env.environment, MultiUserEnvironment):
      raise ValueError('Multi-user agent requires multi-user environment.')
    if not self._agent.multi_user and isinstance(
        self._env.environment, MultiUserEnvironment):
      raise ValueError('Single-user agent requires single-user environment.')
    self._summary_writer.add_graph(graph=tf.get_default_graph())
    self._sess.run(tf.global_variables_initializer())
    self._sess.run(tf.local_variables_initializer())

  def _initialize_checkpointer_and_maybe_resume(self, checkpoint_file_prefix):
    """Reloads the latest checkpoint if it exists.

    This method will first create a `Checkpointer` object and then call
    `checkpointer.get_latest_checkpoint_number` to determine if there is a valid
    checkpoint in self._checkpoint_dir, and what the largest file number is.
    If a valid checkpoint file is found, it will load the bundled data from this
    file and will pass it to the agent for it to reload its data.
    If the agent is able to successfully unbundle, this method will increase and
    return the iteration number keyed by 'current_iteration' and the step number
    keyed by 'total_steps' as the return values.

    Args:
      checkpoint_file_prefix: str, the checkpoint file prefix.
    Returns:
      start_iteration: The iteration number to be continued after the latest
        checkpoint.
      start_step: The step number to be continued after the latest checkpoint.
    """
    self._checkpointer = checkpointer.Checkpointer(self._checkpoint_dir,
                                                   checkpoint_file_prefix)
    start_iteration = 0
    start_step = 0
    # Check if checkpoint exists.
    # Note that the existence of checkpoint 0 means that we have finished
    # iteration 0 (so we will start from iteration 1).
    latest_checkpoint_version = checkpointer.get_latest_checkpoint_number(
        self._checkpoint_dir)
    if latest_checkpoint_version >= 0:
      assert not self._episode_writer, 'Can only log episodes from scratch.'
      experiment_data = self._checkpointer.load_checkpoint(
          latest_checkpoint_version)
      start_iteration = experiment_data['current_iteration'] + 1
      del experiment_data['current_iteration']
      start_step = experiment_data['total_steps'] + 1
      del experiment_data['total_steps']
      if self._agent.unbundle(self._checkpoint_dir, latest_checkpoint_version,
                              experiment_data):
        tf.logging.info(
            'Reloaded checkpoint and will start from '
            'iteration %d', start_iteration)
    return start_iteration, start_step

  def _log_one_step(self, user_obs, doc_obs, slate, responses, reward,
                    is_terminal, sequence_example):
    """Adds one step of agent-environment interaction into SequenceExample.

    Args:
      user_obs: An array of floats representing user state observations
      doc_obs: A list of observations of the documents
      slate: An array of indices to doc_obs
      responses: A list of observations of responses for items in the slate
      reward: A float for the reward returned after this step
      is_terminal: A boolean for whether a terminal state has been reached
      sequence_example: A SequenceExample proto for logging current episode
    """

    def _add_float_feature(feature, values):
      feature.feature.add(float_list=tf.train.FloatList(value=values))

    def _add_int64_feature(feature, values):
      feature.feature.add(int64_list=tf.train.Int64List(value=values))

    if self._episode_writer is None:
      return
    fl = sequence_example.feature_lists.feature_list

    if isinstance(self._env.environment, MultiUserEnvironment):
      for i, (single_user,
              single_slate,
              single_user_responses,
              single_reward) in enumerate(zip(user_obs,
                                              slate,
                                              responses,
                                              reward)):
        user_space = list(self._env.observation_space.spaces['user'].spaces)[i]
        _add_float_feature(fl['user_%d' % i], spaces.flatten(
            user_space, single_user))
        _add_int64_feature(fl['slate_%d' % i], single_slate)
        _add_float_feature(fl['reward_%d' % i], [single_reward])
        for j, response in enumerate(single_user_responses):
          resp_space = self._env.observation_space.spaces['response'][i][0]
          for k in response:
            _add_float_feature(fl['response_%d_%d_%s' % (i, j, k)],
                               spaces.flatten(resp_space, response))
    else:  # single-user environment
      _add_float_feature(
          fl['user'],
          spaces.flatten(self._env.observation_space.spaces['user'], user_obs))
      _add_int64_feature(fl['slate'], slate)
      for i, response in enumerate(responses):
        resp_space = self._env.observation_space.spaces['response'][0]
        for k in response:
          _add_float_feature(fl['response_%d_%s' % (i, k)],
                             spaces.flatten(resp_space, response))
      _add_float_feature(fl['reward'], [reward])

    for i, doc in enumerate(list(doc_obs.values())):
      doc_space = list(
          self._env.observation_space.spaces['doc'].spaces.values())[i]
      _add_float_feature(fl['doc_%d' % i], spaces.flatten(doc_space, doc))

    _add_int64_feature(fl['is_terminal'], [is_terminal])

  def _run_one_episode(self):
    """Executes a full trajectory of the agent interacting with the environment.

    Returns:
      The number of steps taken and the total reward.
    """
    step_number = 0
    total_reward = 0.

    start_time = time.time()

    sequence_example = tf.train.SequenceExample()
    observation = self._env.reset()
    action = self._agent.begin_episode(observation)

    # Keep interacting until we reach a terminal state.
    while True:
      last_observation = observation
      observation, reward, done, info = self._env.step(action)
      self._log_one_step(last_observation['user'], last_observation['doc'],
                         action, observation['response'], reward, done,
                         sequence_example)
      # Update environment-specific metrics with responses to the slate.
      self._env.update_metrics(observation['response'], info)

      total_reward += reward
      step_number += 1

      if done:
        break
      elif step_number == self._max_steps_per_episode:
        # Stop the run loop once we reach the true end of episode.
        break
      else:
        action = self._agent.step(reward, observation)

    self._agent.end_episode(reward, observation)
    if self._episode_writer is not None:
      self._episode_writer.write(sequence_example.SerializeToString())

    time_diff = time.time() - start_time
    self._update_episode_metrics(
        episode_length=step_number,
        episode_time=time_diff,
        episode_reward=total_reward)

    return step_number, total_reward

  def _initialize_metrics(self):
    """Initializes the metrics."""
    self._stats = {
        'episode_length': [],
        'episode_time': [],
        'episode_reward': [],
    }
    # Initialize environment-specific metrics.
    self._env.reset_metrics()

  def _update_episode_metrics(self, episode_length, episode_time,
                              episode_reward):
    """Updates the episode metrics with one episode."""

    self._stats['episode_length'].append(episode_length)
    self._stats['episode_time'].append(episode_time)
    self._stats['episode_reward'].append(episode_reward)

  def _write_metrics(self, step, suffix):
    """Writes the metrics to Tensorboard summaries."""

    def add_summary(tag, value):
      summary = tf.Summary(
          value=[tf.Summary.Value(tag=tag + '/' + suffix, simple_value=value)])
      self._summary_writer.add_summary(summary, step)

    num_steps = np.sum(self._stats['episode_length'])
    time_per_step = np.sum(self._stats['episode_time']) / num_steps

    add_summary('TimePerStep', time_per_step)
    add_summary('AverageEpisodeLength', np.mean(self._stats['episode_length']))
    add_summary('AverageEpisodeRewards', np.mean(self._stats['episode_reward']))
    add_summary('StdEpisodeRewards', np.std(self._stats['episode_reward']))

    # Environment-specific Tensorboard summaries.
    self._env.write_metrics(add_summary)

    self._summary_writer.flush()

  def _checkpoint_experiment(self, iteration, total_steps):
    """Checkpoints experiment data.

    Args:
      iteration: int, iteration number for checkpointing.
      total_steps: int, total number of steps for all iterations so far.
    """
    experiment_data = self._agent.bundle_and_checkpoint(self._checkpoint_dir,
                                                        iteration)
    if experiment_data:
      experiment_data['current_iteration'] = iteration
      experiment_data['total_steps'] = total_steps
      self._checkpointer.save_checkpoint(iteration, experiment_data)


@gin.configurable
class TrainRunner(Runner):
  """Object that handles running the training.

  See main.py for a simple example to train an agent.
  """

  def __init__(self, max_training_steps=250000, num_iterations=100,
               checkpoint_frequency=1, **kwargs):
    tf.logging.info(
        'max_training_steps = %s, number_iterations = %s,'
        'checkpoint frequency = %s iterations.', max_training_steps,
        num_iterations, checkpoint_frequency)

    super(TrainRunner, self).__init__(**kwargs)
    self._max_training_steps = max_training_steps
    self._num_iterations = num_iterations
    self._checkpoint_frequency = checkpoint_frequency

    self._output_dir = os.path.join(self._base_dir, 'train')
    self._checkpoint_dir = os.path.join(self._output_dir, 'checkpoints')

    self._set_up(eval_mode=False)

  def run_experiment(self):
    """Runs a full experiment, spread over multiple iterations."""
    tf.logging.info('Beginning training...')
    start_iter, total_steps = self._initialize_checkpointer_and_maybe_resume(
        self._checkpoint_file_prefix)
    if self._num_iterations <= start_iter:
      tf.logging.warning('num_iterations (%d) < start_iteration(%d)',
                         self._num_iterations, start_iter)
      return

    for iteration in range(start_iter, self._num_iterations):
      tf.logging.info('Starting iteration %d', iteration)
      total_steps = self._run_train_phase(total_steps)
      if iteration % self._checkpoint_frequency == 0:
        self._checkpoint_experiment(iteration, total_steps)

  def _run_train_phase(self, total_steps):
    """Runs training phase and updates total_steps."""

    self._initialize_metrics()

    num_steps = 0

    while num_steps < self._max_training_steps:
      episode_length, _ = self._run_one_episode()
      num_steps += episode_length

    total_steps += num_steps
    self._write_metrics(total_steps, suffix='train')
    return total_steps


@gin.configurable
class EvalRunner(Runner):
  """Object that handles running the evaluation.

  See main.py for a simple example to evaluate an agent.
  """

  def __init__(self,
               max_eval_episodes=125000,
               test_mode=False,
               min_interval_secs=30,
               train_base_dir=None,
               **kwargs):
    tf.logging.info('max_eval_episodes = %s', max_eval_episodes)
    super(EvalRunner, self).__init__(**kwargs)
    self._max_eval_episodes = max_eval_episodes
    self._test_mode = test_mode
    self._min_interval_secs = min_interval_secs

    self._output_dir = os.path.join(self._base_dir,
                                    'eval_%s' % max_eval_episodes)
    tf.io.gfile.makedirs(self._output_dir)
    if train_base_dir is None:
      train_base_dir = self._base_dir
    self._checkpoint_dir = os.path.join(train_base_dir, 'train', 'checkpoints')

    self._set_up(eval_mode=True)

  def run_experiment(self):
    """Runs a full experiment, spread over multiple iterations."""
    tf.logging.info('Beginning evaluation...')
    # Use the checkpointer class.
    self._checkpointer = checkpointer.Checkpointer(
        self._checkpoint_dir, self._checkpoint_file_prefix)
    checkpoint_version = -1
    # Check new checkpoints in a loop.
    while True:
      # Check if checkpoint exists.
      # Note that the existence of checkpoint 0 means that we have finished
      # iteration 0 (so we will start from iteration 1).
      latest_checkpoint_version = checkpointer.get_latest_checkpoint_number(
          self._checkpoint_dir)
      # checkpoints_iterator already makes sure a new checkpoint exists.
      if latest_checkpoint_version <= checkpoint_version:
        time.sleep(self._min_interval_secs)
        continue
      checkpoint_version = latest_checkpoint_version
      experiment_data = self._checkpointer.load_checkpoint(
          latest_checkpoint_version)
      assert self._agent.unbundle(self._checkpoint_dir,
                                  latest_checkpoint_version, experiment_data)

      self._run_eval_phase(experiment_data['total_steps'])
      if self._test_mode:
        break

  def _run_eval_phase(self, total_steps):
    """Runs evaluation phase given model has been trained for total_steps."""

    self._env.reset_sampler()
    self._initialize_metrics()

    num_episodes = 0
    episode_rewards = []

    while num_episodes < self._max_eval_episodes:
      _, episode_reward = self._run_one_episode()
      episode_rewards.append(episode_reward)
      num_episodes += 1

    self._write_metrics(total_steps, suffix='eval')

    output_file = os.path.join(self._output_dir, 'returns_%s' % total_steps)
    tf.logging.info('eval_file: %s', output_file)
    with tf.io.gfile.GFile(output_file, 'w+') as f:
      f.write(str(episode_rewards))

In [ ]:
"""Utility functions for RecSim environment."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


def aggregate_video_cluster_metrics(responses, metrics, info=None):
  """Aggregates the video cluster metrics with one step responses.

  Args:
    responses: a dictionary of names, observed responses.
    metrics: A dictionary mapping from metric_name to its value in float.
    info: Additional info for computing metrics (ignored here)

  Returns:
    A dictionary storing metrics after aggregation.
  """
  del info  # Unused.
  is_clicked = False
  metrics['impression'] += 1

  for response in responses:
    if not response['click']:
      continue
    is_clicked = True
    metrics['click'] += 1
    metrics['quality'] += response['quality']
    cluster_id = response['cluster_id']
    metrics['cluster_watch_count_cluster_%d' % cluster_id] += 1

  if not is_clicked:
    metrics['cluster_watch_count_no_click'] += 1
  return metrics


def write_video_cluster_metrics(metrics, add_summary_fn):
  """Writes average video cluster metrics using add_summary_fn."""
  add_summary_fn('CTR', metrics['click'] / metrics['impression'])
  if metrics['click'] > 0:
    add_summary_fn('AverageQuality', metrics['quality'] / metrics['click'])
  for k in metrics:
    prefix = 'cluster_watch_count_cluster_'
    if k.startswith(prefix):
      add_summary_fn('cluster_watch_count_frac/cluster_%s' % k[len(prefix):],
                     metrics[k] / metrics['impression'])
  add_summary_fn(
      'cluster_watch_count_frac/no_click',
      metrics['cluster_watch_count_no_click'] / metrics['impression'])

In [ ]:
env_config = {'slate_size': 2,
              'seed': 0,
              'num_candidates': 15,
              'resample_documents': True}
ie_environment = create_environment(env_config)

In [ ]:
initial_observation = ie_environment.reset()

In [ ]:
print('User Observable Features')
print(initial_observation['user'])
print('User Response')
print(initial_observation['response'])
print('Document Observable Features')
for doc_id, doc_features in initial_observation['doc'].items():
  print('ID:', doc_id, 'features:', doc_features)

User Observable Features
[]
User Response
None
Document Observable Features
ID: 15 features: {'quality': array(1.22720163), 'cluster_id': 1}
ID: 16 features: {'quality': array(1.29258489), 'cluster_id': 1}
ID: 17 features: {'quality': array(1.23977078), 'cluster_id': 1}
ID: 18 features: {'quality': array(1.46045555), 'cluster_id': 1}
ID: 19 features: {'quality': array(2.10233425), 'cluster_id': 0}
ID: 20 features: {'quality': array(1.09572905), 'cluster_id': 1}
ID: 21 features: {'quality': array(2.37256963), 'cluster_id': 0}
ID: 22 features: {'quality': array(1.34928002), 'cluster_id': 1}
ID: 23 features: {'quality': array(1.00670188), 'cluster_id': 1}
ID: 24 features: {'quality': array(1.20448562), 'cluster_id': 1}
ID: 25 features: {'quality': array(2.18351159), 'cluster_id': 0}
ID: 26 features: {'quality': array(1.19411585), 'cluster_id': 1}
ID: 27 features: {'quality': array(1.03514646), 'cluster_id': 1}
ID: 28 features: {'quality': array(2.29592623), 'cluster_id': 0}
ID: 29 feature

In [ ]:
print('Document observation space')
for key, space in ie_environment.observation_space['doc'].spaces.items():
  print(key, ':', space)
print('Response observation space')
print(ie_environment.observation_space['response'])
print('User observation space')
print(ie_environment.observation_space['user'])

Document observation space
15 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf, (), float32))
16 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf, (), float32))
17 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf, (), float32))
18 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf, (), float32))
19 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf, (), float32))
20 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf, (), float32))
21 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf, (), float32))
22 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf, (), float32))
23 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf, (), float32))
24 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf, (), float32))
25 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf, (), float32))
26 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf, (), float32))
27 : Dict('cluster_id': Discrete(2), 'quality': Box(0.0, inf,

In [ ]:
slate = [0, 1]
for slate_doc in slate:
  print(list(initial_observation['doc'].items())[slate_doc])

('15', {'quality': array(1.22720163), 'cluster_id': 1})
('16', {'quality': array(1.29258489), 'cluster_id': 1})


In [ ]:
ie_environment.action_space

MultiDiscrete([15 15])

In [ ]:
observation, reward, done, _ = ie_environment.step(slate)

In [ ]:
"""Abstract interface for recommender system agents."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
from absl import logging
import six


@six.add_metaclass(abc.ABCMeta)
class AbstractRecommenderAgent(object):
  """Abstract class to model a recommender system agent."""
  _multi_user = False

  def __init__(self, action_space):
    """Initializes AbstractRecommenderAgent.

    Args:
      action_space: A gym.spaces object that specifies the format of actions.
    """
    self._slate_size = action_space.nvec.shape[0]

  @property
  def multi_user(self):
    """Returns boolean indicating whether this agent serves multiple users."""
    return self._multi_user

  @abc.abstractmethod
  def step(self, reward, observation):
    """Records the most recent transition and returns the agent's next action.

    We store the observation of the last time step since we want to store it
    with the reward.

    Args:
      reward: The reward received from the agent's most recent action as a
        float.
      observation: A dictionary that includes the most recent observations.

    Returns:
      slate: An integer array of size _slate_size, where each element is an
        index into the list of doc_obs
    """

  @abc.abstractmethod
  def bundle_and_checkpoint(self, checkpoint_dir, iteration_number):
    """Returns a self-contained bundle of the agent's state.

    This is used for checkpointing. It will return a dictionary containing all
    non-TensorFlow objects (to be saved into a file by the caller), and it saves
    all TensorFlow objects into a checkpoint file.

    Args:
      checkpoint_dir: A string for the directory where objects will be saved.
      iteration_number: An integer of iteration number to use for naming the
        checkpoint file.

    Returns:
      A dictionary containing additional Python objects to be checkpointed by
        the experiment. Each key is a string for the object name and the value
        is actual object. If the checkpoint directory does not exist, returns
        empty dictionary.
    """

  @abc.abstractmethod
  def unbundle(self, checkpoint_dir, iteration_number, bundle_dict):
    """Restores the agent from a checkpoint.

    Restores the agent's Python objects to those specified in bundle_dict,
    and restores the TensorFlow objects to those specified in the
    checkpoint_dir. If the checkpoint_dir does not exist, will not reset the
    agent's state.

    Args:
      checkpoint_dir: A string that represents the path to the checkpoint saved
        by tf.Save.
      iteration_number: An integer that represents the checkpoint version and is
        used when restoring replay buffer.
      bundle_dict: A dict containing additional Python objects owned by the
        agent. Each key is an object name and the value is the actual object.

    Returns:
      bool, True if unbundling was successful.
    """


class AbstractEpisodicRecommenderAgent(AbstractRecommenderAgent):
  """Abstract class for recommender systems that solves episodic tasks."""

  def __init__(self, action_space, summary_writer=None):
    """Initializes AbstractEpisodicRecommenderAgent.

    Args:
      action_space: A gym.spaces object that specifies the format of actions.
      summary_writer: A Tensorflow summary writer to pass to the agent
        for in-agent training statistics in Tensorboard.
    """
    super(AbstractEpisodicRecommenderAgent, self).__init__(action_space)
    self._episode_num = 0
    self._summary_writer = summary_writer

  def begin_episode(self, observation=None):
    """Returns the agent's first action for this episode.

    Args:
      observation: numpy array, the environment's initial observation.

    Returns:
      slate: An integer array of size _slate_size, where each element is an
        index into the list of doc_obs
    """
    self._episode_num += 1
    return self.step(0, observation)

  def end_episode(self, reward, observation=None):
    """Signals the end of the episode to the agent.

    Args:
      reward: An float that is the last reward from the environment.
      observation: numpy array that represents the last observation of the
        episode.
    """
    pass

  def bundle_and_checkpoint(self, checkpoint_dir, iteration_number):
    """Returns a self-contained bundle of the agent's state.

    Args:
      checkpoint_dir: A string that represents the path to the checkpoint and is
        used when we save TensorFlow objects by tf.Save.
      iteration_number: An integer that represents the checkpoint version and is
        used when restoring replay buffer.

    Returns:
      A dictionary containing additional Python objects to be checkpointed by
        the experiment. Each key is a string for the object name and the value
        is actual object. If the checkpoint directory does not exist, returns
        empty dictionary.
    """
    del checkpoint_dir  # Unused.
    del iteration_number  # Unused.
    bundle_dict = {}
    bundle_dict['episode_num'] = self._episode_num
    return bundle_dict

  def unbundle(self, checkpoint_dir, iteration_number, bundle_dict):
    """Restores the agent from a checkpoint.

    Args:
      checkpoint_dir: A string that represents the path to the checkpoint and is
        used when we save TensorFlow objects by tf.Save.
      iteration_number: An integer that represents the checkpoint version and is
        used when restoring replay buffer.
      bundle_dict: A dict containing additional Python objects owned by the
        agent. Each key is an object name and the value is the actual object.

    Returns:
      bool, True if unbundling was successful.
    """
    del checkpoint_dir  # Unused.
    del iteration_number  # Unused.
    if 'episode_num' not in bundle_dict:
      logging.warning(
          'Could not unbundle from checkpoint files with exception.')
      return False
    self._episode_num = bundle_dict['episode_num']
    return True


@six.add_metaclass(abc.ABCMeta)
class AbstractMultiUserEpisodicRecommenderAgent(
    AbstractEpisodicRecommenderAgent):
  """Abstract class to model a recommender agent handling multiple users."""
  _multi_user = True

  def __init__(self, action_space):
    """Initializes AbstractMultiUserEpisodicRecommenderAgent.

    Args:
      action_space: A gym.spaces object that specifies the format of actions.
    """
    self._num_users = len(action_space)
    if not self._num_users > 0:
      raise ValueError('Multi-user agent must have at least 1 user.')
    super(AbstractMultiUserEpisodicRecommenderAgent, self).__init__(
        action_space[0]
    )


class AbstractHierarchicalAgentLayer(AbstractRecommenderAgent):
  """Parent class for stackable agent layers."""

  def __init__(self, action_space, *base_agent_ctors):
    super(AbstractHierarchicalAgentLayer, self).__init__(action_space)
    self._base_agent_ctors = base_agent_ctors
    self._base_agents = None

  def _preprocess_reward_observation(self, reward, observation):
    r"""Modifies the reward and observation before passing to base agent.

    This function is used to modify the observation and reward before
    propagating it downward to the base agent. For example, it can
    inject additional features like sufficient statistics by inserting fields
    to observation[\'user\'], or, to implement regularization schemes by
    subtracting penalties from the reward.

    Args:
      reward: float number.
      observation: gym space in recsim format.

    Returns:
      reward: float number.
      observation: gym space in recsim format.
    """
    return reward, observation

  @abc.abstractmethod
  def _postprocess_actions(self, action_list):
    r"""Aggregates (possibly abstract) base agent actions into a valid slate."""

  def begin_episode(self, observation=None):
    if observation is not None:
      _, observation = self._preprocess_reward_observation(0, observation)
    action_list = [
        base_agent.begin_episode(observation=observation)
        for base_agent in self._base_agents
    ]
    return self._postprocess_actions(action_list)

  def end_episode(self, reward, observation):
    reward, observation = self._preprocess_reward_observation(
        reward, observation)
    action_list = [
        base_agent.end_episode(reward, observation=observation)
        for base_agent in self._base_agents
    ]
    return self._postprocess_actions(action_list)

  def bundle_and_checkpoint(self, checkpoint_dir, iteration_number):
    """Returns a self-contained bundle of the agent's state.

    Args:
      checkpoint_dir: A string for the directory where objects will be saved.
      iteration_number: An integer of iteration number to use for naming the
        checkpoint file.

    Returns:
      A dictionary containing additional Python objects to be checkpointed by
        the experiment. Each key is a string for the object name and the value
        is actual object. If the checkpoint directory does not exist, returns
        empty dictionary.
    """
    bundle_dict = {}
    for i, base_agent in enumerate(self._base_agents):
      base_bundle_dict = base_agent.bundle_and_checkpoint(
          checkpoint_dir, iteration_number)
      bundle_dict['base_agent_bundle_{}'.format(i)] = base_bundle_dict
    return bundle_dict

  def unbundle(self, checkpoint_dir, iteration_number, bundle_dict):
    """Restores the agent from a checkpoint.

    Args:
      checkpoint_dir: A string that represents the path to the checkpoint saved
        by tf.Save.
      iteration_number: An integer that represents the checkpoint version and is
        used when restoring replay buffer.
      bundle_dict: A dict containing additional Python objects owned by the
        agent. Each key is an object name and the value is the actual object.

    Returns:
      bool, True if unbundling was successful.
    """
    success = True
    for i, base_agent in enumerate(self._base_agents):
      if 'base_agent_bundle_{}'.format(i) not in bundle_dict:
        logging.warning('Base agent bundle not found in bundle.')
        return False
      success &= base_agent.unbundle(
          checkpoint_dir, iteration_number,
          bundle_dict['base_agent_bundle_{}'.format(i)])
    return success

In [ ]:
"""Helper classes to record user response history on recommendations."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
from gym import spaces



class SufficientStatisticsLayer(AbstractHierarchicalAgentLayer):
  """A module to log user responses on different clusters.

  This module assumes each document belongs to single cluster and we know the
  number of possible clusters. Every time we increase impression count for a
  cluster if the agent recommends a document from that cluster. We also increase
  click count for a cluster if user responds a click.
  """

  def __init__(self, base_agent_ctor, observation_space, action_space,
               sufficient_statistics_space, **kwargs):
    """Initializes a UserClusterHistory object.

    Args:
      base_agent_ctor: a constructor for the base agent.
      observation_space: a gym.spaces object specifying the format of
        observations.
      action_space: A gym.spaces object that specifies the format of actions.
      sufficient_statistics_space: a gym.spaces object specifying the format of
        the created sufficient statistics.
      **kwargs: arguments to pass to the downstream agent at construction time.
    """
    super(SufficientStatisticsLayer, self).__init__(action_space,
                                                    base_agent_ctor)
    self._sufficient_statistcs_space = sufficient_statistics_space
    self._sufficient_statistics = None
    augmented_observation_space = {
        'user':
            spaces.Dict({
                'raw_observation': observation_space.spaces['user'],
                'sufficient_statistics': sufficient_statistics_space
            }),
        'response':
            observation_space.spaces['response'],
        'doc':
            observation_space.spaces['doc']
    }
    self._observation_space = observation_space
    self._base_observation_space = spaces.Dict(augmented_observation_space)
    kwargs['observation_space'] = self._base_observation_space
    kwargs['action_space'] = action_space
    self._base_agents = [
        self._base_agent_ctors[0](**kwargs),
    ]
    self._reset()

  @property
  def observation_space(self):
    return self._observation_space

  @abc.abstractmethod
  def _update(self, observation):
    """Updates self._sufficient_statistics given a new observation.

    If self._sufficient_statistics is None, this function must also initialize
    it.

    Args:
     observation: an observation conforming to self._observation_space.
    """

  @abc.abstractmethod
  def _create_observation(self):
    """Formats self._sufficient_statistics into an observation."""

  def _preprocess_reward_observation(self, reward, observation):
    self._update(observation)
    augmented_observation = {key: value for key, value in observation.items()}
    augmented_observation['user'] = {
        'raw_observation': augmented_observation['user'],
        'sufficient_statistics': self._create_observation()
    }
    return reward, augmented_observation

  def _postprocess_actions(self, action_list):
    # Does not modify the action of the base agent.
    return action_list[0]

  def step(self, reward, observation):
    reward, augmented_observation = self._preprocess_reward_observation(
        reward, observation)
    action_list = [
        base_agent.step(reward, augmented_observation)
        for base_agent in self._base_agents
    ]
    return self._postprocess_actions(action_list)

  def end_episode(self, reward, observation):
    super(SufficientStatisticsLayer, self).end_episode(reward, observation)
    self._reset()

  def _reset(self):
    """Resets the memory for a new user."""
    self._sufficient_statistics = None

In [ ]:
"""Helper class to collect cluster click and impression counts."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from gym import spaces
import numpy as np



class ClusterClickStatsLayer(SufficientStatisticsLayer):
  """Track impressions and clicks on a per-cluster basis and pass down to agent.

  This module assumes each document belongs to single cluster and we know the
  number of possible clusters. Every time we increase impression count for a
  cluster if the agent recommends a document from that cluster. We also increase
  click count for a cluster if user responds a click.
  """

  def __init__(self, base_agent_ctor, observation_space, action_space,
               **kwargs):
    """Initializes a ClusterClickStatsLayer object.

    Args:
      base_agent_ctor: a constructor for the base agent.
      observation_space: a gym.spaces object specifying the format of
        observations.
      action_space: A gym.spaces object that specifies the format of actions.
      **kwargs: arguments to pass to the downstream agent at construction time.
    """
    single_response_space = observation_space.spaces['response'].spaces[0]
    if 'cluster_id' not in single_response_space.spaces:
      raise ValueError('observation_space.spaces[\'response\']'
                       ' must contain \'cluster_id\' key.')
    cluster_id_space = single_response_space.spaces['cluster_id']
    if isinstance(cluster_id_space, spaces.Box):
      if len(cluster_id_space.high) > 1:
        raise ValueError('cluster_id response field must be 0 dimensional.')
      num_clusters = cluster_id_space.high
    elif isinstance(cluster_id_space, spaces.Discrete):
      num_clusters = cluster_id_space.n
    else:
      raise ValueError('cluster_id response field must be either gym.spaces.Box'
                       ' or gym spaces.Discrete')
    self._num_clusters = num_clusters
    if 'click' not in single_response_space.spaces:
      raise ValueError(
          'observation_space.spaces[\'response\'] must contain \'click\' key.')
    suf_stat_space = spaces.Dict({
        'impression_count':
            spaces.Box(
                shape=(num_clusters,), dtype=np.float32, low=0.0, high=np.inf),
        'click_count':
            spaces.Box(
                shape=(num_clusters,), dtype=np.float32, low=0.0, high=np.inf)
    })
    super(ClusterClickStatsLayer,
          self).__init__(base_agent_ctor, observation_space, action_space,
                         suf_stat_space, **kwargs)

  def _create_observation(self):
    return {
        'impression_count':
            np.array(self._sufficient_statistics['impression_count']),
        'click_count':
            np.array(self._sufficient_statistics['click_count']),
    }

  def _update(self, observation):
    """Updates user impression/click count given user response on each item."""
    if self._sufficient_statistics is None:
      self._sufficient_statistics = {
          'impression_count': [
              0,
          ] * self._num_clusters,
          'click_count': [
              0,
          ] * self._num_clusters
      }
    if observation['response'] is not None:
      for response in observation['response']:
        cluster_id = int(response['cluster_id'])
        self._sufficient_statistics['impression_count'][cluster_id] += 1
        if response['click']:
          self._sufficient_statistics['click_count'][cluster_id] += 1

In [ ]:
"""Classes for Bandit Algorithms."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np


class MABAlgorithm(object):
  """Base class for Multi-armed bandit (MAB) algorithms.

  We implement multi-armed bandit algorithms with confidence width tuning
  proposed in Hsu et al. https://arxiv.org/abs/1904.02664.

  Attributes:
    pulls: A numpy array which counts number of pulls of each arm
    reward: A numpy array which sums up reward of each arm
    optimism_scaling: A float specifying the confidence level. Default value
      (1.0) corresponds to the exploration strategy presented in the literature.
      A smaller number means less exploration and more exploitation.
    _rng: An instance of random.RandomState for random number generation
  """

  def __init__(self, num_arms, params, seed=0):
    """Initializes MABAlgorithm.

    Args:
      num_arms: Number of arms. Must be greater than one.
      params: A dictionary which includes additional parameters like
        optimism_scaling. Default is an empty dictionary.
      seed: Random seed for this object. Default is zero.
    """
    if num_arms < 2:
      raise ValueError('num_arms must be greater than one.')
    self.pulls = np.zeros(num_arms)
    self.reward = np.zeros(num_arms)
    self._rng = np.random.RandomState(seed)

    self.optimism_scaling = 1.0
    for attr, val in params.items():
      setattr(self, attr, val)

  def set_state(self, pulls, reward):
    if len(pulls) != len(self.pulls) or len(reward) != len(self.reward):
      raise ValueError('Cannot set state with a different number of arms.')
    self.pulls[:] = pulls
    self.reward[:] = reward

  def update(self, arm, reward):
    if reward < 0 or reward > 1:
      raise ValueError('reward must be in [0, 1].')
    self.pulls[arm] += 1
    self.reward[arm] += reward


class UCB1(MABAlgorithm):
  """UCB1 algorithm.

  See "Finite-time Analysis of the Multiarmed Bandit Problem" by Auer,
  Cesa-Bianchi, and Fischer.
  """

  def get_score(self, t):
    """Computes upper confidence bounds of reward / pulls at round t."""
    # Pull any arm that we haven't pulled.
    if not all(self.pulls):
      return np.where(self.pulls > 0, 0, np.Inf)
    ct = self.optimism_scaling * np.sqrt(2 * np.log(t))
    return self.reward / self.pulls + ct * np.sqrt(1 / self.pulls)

  def get_arm(self, t):
    return np.argmax(self.get_score(t))

  @staticmethod
  def print():
    return 'UCB1'


class KLUCB(MABAlgorithm):
  """Kullback-Leibler Upper Confidence Bounds (KL-UCB) algorithm.

  See "The KL-UCB algorithm for bounded stochastic bandits and beyond" by
  Garivier and Cappe.
  """

  def get_score(self, t):
    """Computes upper confidence bounds of reward / pulls at round t."""
    # Pull any arm that we haven't pulled.
    if not all(self.pulls):
      return np.where(self.pulls > 0, 0, np.Inf)
    c = self.optimism_scaling**2 * (np.log(t) +
                                    3 * np.log(np.log(t))) / self.pulls
    p = self.reward / self.pulls

    # KL-divergence d(p, q) is strictly increasing over [p, 1].
    # Use binary search to find q such that d(p, q) <= c.
    qmin = p
    qmax = np.ones(p.size)
    for _ in range(16):  # Error bounded by 2^-16.
      q = (qmax + qmin) / 2
      ndx = (np.where(p > 0, p * np.log(p / q), 0) +
             np.where(p < 1, (1 - p) * np.log((1 - p) / (1 - q)), 0)) < c
      qmin[ndx] = q[ndx]
      qmax[~ndx] = q[~ndx]

    return q

  def get_arm(self, t):
    return np.argmax(self.get_score(t))

  @staticmethod
  def print():
    return 'KL-UCB'


class ThompsonSampling(MABAlgorithm):
  """Thompson Sampling algorithm for the Bernoulli bandit.

  See "Further Optimal Regret Bounds for Thompson Sampling" by Agrawal and
    Goyal.
  """

  def update(self, arm, reward):
    if reward > 0 and reward < 1:
      reward = float(self._rng.rand() < reward)
    MABAlgorithm.update(self, arm, reward)

  def get_score(self, t):
    """Samples scores from the posterior distribution."""
    del t
    # Generate a beta distribution based on the expectation of reward.
    alpha = 1 + self.reward / self.optimism_scaling**2
    beta = 1 + (self.pulls - self.reward) / self.optimism_scaling**2
    return self._rng.beta(alpha, beta)

  def get_arm(self, t):
    return np.argmax(self.get_score(t))

  @staticmethod
  def print():
    return 'ThompsonSampling'

In [ ]:
tmp_base_dir = 'recsim/recsim/'
runner = TrainRunner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_agent,
    env=create_environment(env_config),
    episode_log_file="",
    max_training_steps=50,
    num_iterations=10)
runner.run_experiment()

In [ ]:
import pandas as pd

# Load the MovieLens-1M dataset
ratings_df = pd.read_csv('amazon_book_interaction_rating.csv')
movies_df = pd.read_csv('amazon_book_item.csv')
users_df = pd.read_csv('user_profiles.csv')
user = users_df[['id', 'name', 'gender', 'age']]
user = user.rename(columns={'id': 'user_id'})
movies_df = movies_df.rename(columns={'id': 'item_id'})


# Compute average rating per movie
avg_ratings = ratings_df.groupby('item_id')['rating'].mean().to_dict()

# Merge datasets
data = ratings_df.merge(movies_df, on="item_id").merge(user_reduced, on="user_id")

In [ ]:
# Extract unique single-word genres
unique_genres = {
    word.strip().lower()
    for genres in movies_df['genre'].dropna()
    for genre in genres
    for word in genre.replace('-', ' ').split()   # split by space and hyphen
    if word.strip() != ''
}

# Map each genre (word) to an index
genre_to_index = {genre: idx for idx, genre in enumerate(sorted(unique_genres))}
num_genres = len(unique_genres)

# print(num_genres)

In [ ]:
# Merge the two datasets based on the 'lister_music_id' column
user_character = pd.merge(ratings_df, movies_df, on='item_id', how='left')

# Filter the dataset for ratings 4 and 5
filtered_df = user_character[user_character["rating"].isin([4, 5])]

# Group by user_id and collect all genres into a single list
user_genres = (
    filtered_df.groupby("user_id")["genre"]
    .apply(lambda x: [g for sublist in x for g in sublist])  # flatten lists
    .reset_index()
)

# Deduplicate and sort genres per user
user_genres["genre"] = user_genres["genre"].apply(
    lambda x: "|".join(sorted(set(x)))
)

# print(user_genres.head(10))

In [ ]:
import logging
from typing import Any, Dict, Optional, List, Tuple
from yacs.config import CfgNode
import os
import math


# logger
def set_logger(log_file, name="default"):
    """
    Set logger.
    Args:
        log_file (str): log file path
        name (str): logger name
    """

    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)

    formatter = logging.Formatter(
        "%(asctime)s - %(levelname)s - %(module)s - %(funcName)s - %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
    )
    print(formatter.format(logging.LogRecord(
    name='root',
    level=logging.INFO,
    pathname=None,
    lineno=None,
    msg='Test message',
    args=None,
    exc_info=None,
    )))
    output_folder = "output"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Create the 'log' folder if it doesn't exist
    log_folder = os.path.join(output_folder, "log")
    if not os.path.exists(log_folder):
        os.makedirs(log_folder)

    # Create the 'message' folder if it doesn't exist
    message_folder = os.path.join(output_folder, "message")
    if not os.path.exists(message_folder):
        os.makedirs(message_folder)
    log_file = os.path.join(log_folder, log_file)
    handler = logging.FileHandler(log_file, mode="w")
    handler.setLevel(logging.INFO)
    handler.setFormatter(formatter)
    logger.handlers = []
    logger.addHandler(handler)
    # print(logger)
    return logger

def load_cfg(cfg_file: str, new_allowed: bool = True) -> CfgNode:
    """
    Load config from file.
    Args:
        cfg_file (str): config file path
        new_allowed (bool): whether to allow new keys in config
    """
    with open(cfg_file, "r") as fi:
        cfg = CfgNode.load_cfg(fi)
    cfg.set_new_allowed(new_allowed)
    return cfg

def add_variable_to_config(cfg: CfgNode, name: str, value: Any) -> CfgNode:
    """
    Add variable to config.
    Args:
        cfg (CfgNode): config
        name (str): variable name
        value (Any): variable value
    """
    cfg.defrost()
    cfg[name] = value
    cfg.freeze()
    return cfg

def ensure_dir(dir_path):
    """
    Make sure the directory exists, if it does not exist, create it
    Args:
        dir_path (str): The directory path.
    """
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

def calculate_entropy(movie_types):
    type_freq = {}
    for movie_type in movie_types:
        if movie_type in type_freq:
            type_freq[movie_type] += 1
        else:
            type_freq[movie_type] = 1

    total_movies = len(movie_types)

    entropy = 0
    for key in type_freq:
        prob = type_freq[key] / total_movies
        entropy -= prob * math.log2(prob)

    return entropy


def get_entropy(inters, data):
    genres = data.get_genres_by_id(inters)
    entropy = calculate_entropy(genres)
    return entropy

In [ ]:
import csv
import os
import math
import numpy as np

class Data:
    """
    Data class for loading data from local files.
    """

    def __init__(self, config):
        self.config = config
        self.items = {}
        self.users = {}
        self.db = None
        self.tot_relationship_num = 0
        self.netwerk_density = 0.0
        self.role_id = -1
        self.interrating = {}
        self.user_ratings = {}
        self.item_ratings = {}
        self.load_items(config["item_path"])
        self.load_users(config["user_path"])
        self.load_interactions_rating(config["rating_path"])

    def load_items(self, file_path):
        """
        Load items from local file.
        """
        with open(file_path, "r", newline="") as file:
            reader = csv.reader(file)
            next(reader)  # Skip the header line
            for row in reader:
                item_id, title, genre, description = row
                self.items[int(item_id)] = {
                    "name": title.strip(),
                    "genre": genre,
                    "description": description.strip(),
                    "inter_cnt": 0,
                    "mention_cnt": 0,
                }

    def load_users(self, file_path):
        """
        Load users from local file.
        """
        cnt = 1
        with open(file_path, "r", newline="") as file:
            reader = csv.reader(file)
            next(reader)  # Skip the header line
            for row in reader:
                # print(len(row), row)
                user_id, name, gender, age, traits, status, interest, feature = row
                self.users[cnt] = {
                    "name": name,
                    "gender": gender,
                    "age": int(age),
                    "traits": traits,
                    "status": status,
                    "interest": interest,
                    "feature": feature,
                }

                cnt += 1



    def load_interactions_rating(self, file_path):
      """
      Load user-item interactions (with rating) from local file.
      Stores in self.interrating as a dict:
      {user_id: [(item_id, rating), ...], ...}
      """
      with open(file_path, "r", newline="") as file:
        reader = csv.reader(file)
        header = next(reader)  # Skip the header line
        for row in reader:
            user_id, item_id, rating = row
            user_id = int(user_id)
            item_id = int(item_id)
            rating = int(rating)
            if user_id not in self.interrating:
                self.interrating[user_id] = []
            self.interrating[user_id].append((item_id, rating))

            if user_id not in self.user_ratings:
                self.user_ratings[user_id] = []
            self.user_ratings[user_id].append(rating)

            # Store item rating
            if item_id not in self.item_ratings:
                self.item_ratings[item_id] = []
            self.item_ratings[item_id].append(rating)

      # Compute and store average historical rating for each user
      self.user_avg_rating = {uid: sum(ratings)/len(ratings) for uid, ratings in self.user_ratings.items() if ratings}

      # Compute and store average historical rating for each item
      self.item_avg_rating = {iid: sum(ratings)/len(ratings) for iid, ratings in self.item_ratings.items() if ratings}


    def get_full_items(self):
        return list(self.items.keys())

    def get_inter_popular_items(self):
        """
        Get the most popular items based on the number of interactions.
        """
        ids = sorted(
            self.items.keys(), key=lambda x: self.items[x]["inter_cnt"], reverse=True
        )[:3]
        return self.get_item_names(ids)

    def add_inter_cnt(self, item_names):
        item_ids = self.get_item_ids(item_names)
        # print("item ids:", item_ids)
        for item_id in item_ids:
            self.items[item_id]["inter_cnt"] += 1

    def add_mention_cnt(self, item_names):
        item_ids = self.get_item_ids(item_names)
        for item_id in item_ids:
            self.items[item_id]["mention_cnt"] += 1

    def get_mention_popular_items(self):
        """
        Get the most popular items based on the number of mentions.
        """
        ids = sorted(
            self.items.keys(), key=lambda x: self.items[x]["mention_cnt"], reverse=True
        )[:3]
        return self.get_item_names(ids)

    def get_item_names(self, item_ids):
        return ["<" + self.items[item_id]["name"] + ">" for item_id in item_ids]

    def get_item_ids(self, item_names):
        item_ids = []
        for item in item_names:
            for item_id, item_info in self.items.items():
                if item_info["name"] in item:
                    item_ids.append(item_id)
                    break
        return item_ids

    def get_item_ids_exact(self, item_names):
        """
        Get item ids from item names.
        I coundn't find any difference with the get_item_ids(item_names) function
        """
        item_ids = []
        for item in item_names:
            for item_id, item_info in self.items.items():
                if item_info["name"] == item:
                    item_ids.append(item_id)
                    break
        return item_ids

    def get_full_users(self):
        return list(self.users.keys())

    def get_user_profile(self, user_id):
        """
        Return the user profile as a formatted string for the given user_id.
        """
        user = self.users.get(user_id)
        if not user:
           return f"User ID {user_id} not found."

        profile = (
        f"User ID: {user_id};; Name: {user['name']}\n"
        f"Gender: {user['gender']};; Age: {user['age']};; Status: {user['status']}\n"
        f"Traits: {user['traits']};; Interest: {user['interest']};; Feature: {user['feature']}\n")
        # f"Positive Behavior: {user['positive_behavior']};; Negative Behavior: {user['negative_behavior']}")
        return profile

    def get_user_names(self, user_ids):
        return [self.users[user_id]["name"] for user_id in user_ids]

    def get_user_ids(self, user_names):
        user_ids = []
        for user in user_names:
            for user_id, user_info in self.users.items():
                if user_info["name"] == user:
                    user_ids.append(user_id)
                    break
        return user_ids

    def get_user_num(self):
        """
        Return the number of users.
        """
        return len(self.users.keys())

    def get_item_num(self):
        """
        Return the number of items.
        """
        return len(self.items.keys())

    def search_items(self, item, k=50):
        """
        Search similar items from faiss db.
        Args:
            item: str, item name
            k: int, number of similar items to return
        """
        docs = self.db.similarity_search(item, k)
        item_names = [doc.page_content for doc in docs]
        return item_names


    def get_item_description_by_id(self, item_ids):
        """
        Get description of items by item id.
        """
        return [self.items[item_id]["description"] for item_id in item_ids]

    def get_item_description_by_name(self, item_names):
        """
        Get description of items by item name.
        """
        item_descriptions = []
        for item in item_names:
            found = False
            for item_id, item_info in self.items.items():
                if item_info["name"] == item.strip(" <>"):
                    item_descriptions.append(item_info["description"])
                    found = True
                    break
            if not found:
                item_descriptions.append("")
        return item_descriptions

    def get_genres_by_id(self, item_ids):
        """
        Get genre of items by item id.
        """
        # return [self.items[item_id]["genre"] for item_id in item_ids]
        return [
            genre
            for item_id in item_ids
            for genre in self.items[item_id]["genre"].split('|')
        ]

    def hit_at_k(self, ground_truth, predicted, k):
        """Return 1 if any of the top-k predicted are relevant, else 0."""
        return int(bool(set(ground_truth) & set(predicted[:k])))

    def ndcg_at_k(self, ground_truth, predicted, k):
        """Compute NDCG@k for a single user."""
        def dcg(rel):
          return np.sum([(2**r - 1) / np.log2(i + 2) for i, r in enumerate(rel)])

        rel = [1 if item in ground_truth else 0 for item in predicted[:k]]
        ideal_rel = sorted([1]*min(len(ground_truth), k) + [0]*(k - min(len(ground_truth), k)), reverse=True)
        dcg_score = dcg(rel)
        idcg_score = dcg(ideal_rel)
        return dcg_score / idcg_score if idcg_score > 0 else 0.0

    def mse(self, ground_truth_ratings, predicted_ratings, items=None):
        """Compute MSE for ratings (on items in both sets)."""
        if items is None:
           items = set(ground_truth_ratings.keys()) & set(predicted_ratings.keys())
        else:
           items = set(items) & set(ground_truth_ratings.keys()) & set(predicted_ratings.keys())
        if not items:
           return np.nan
        errors = [(ground_truth_ratings[i] - predicted_ratings[i]) ** 2 for i in items]
        return np.mean(errors)

    def rmse(self, ground_truth_ratings, predicted_ratings, items=None):
        """Compute RMSE for ratings."""
        return np.sqrt(self.mse(ground_truth_ratings, predicted_ratings, items))

    def safe_log(self, x):
        """Numerically safe log."""
        return math.log(max(x, 1e-15))

    def ordered_probit_probs(self, pred_int, K, taus=None):
        """
        Compute ordered probit class probabilities for a predicted integer rating.

        pred_int : int
            The predicted integer rating (e.g., 1..K).
        K : int
            Number of rating classes (e.g., 5 for 1–5 stars).
        taus : list or array, optional
            Thresholds separating the ordered categories.
            If None, uses equally spaced thresholds [1.5, 2.5, ..., K-0.5].
        """

        if taus is None:
           taus = np.array([1.5 + i for i in range(K-1)])  # default thresholds

        assert len(taus) == K-1

        def Phi(z):
            return 0.5 * (1.0 + math.erf(z / math.sqrt(2.0)))  # Normal CDF

        probs = []
        for k in range(1, K+1):
           if k == 1:
              lower = -np.inf
              upper = taus[0]
           elif k == K:
              lower = taus[-1]
              upper = np.inf
           else:
              lower = taus[k-2]
              upper = taus[k-1]
           p_lower = 0.0 if lower == -np.inf else Phi((lower - pred_int))
           p_upper = 1.0 if upper == np.inf else Phi((upper - pred_int))
           probs.append(max(p_upper - p_lower, 1e-15))

        probs = np.array(probs)
        probs /= probs.sum()  # normalize
        return probs

In [ ]:
class BaseModel(object):
    """Base class for all models."""

    def __init__(self, config, n_users, n_items):
        self.config = config
        self.items = None

    def get_full_sort_items(self, user_id, *args, **kwargs):
        """Get a list of sorted items for a given user."""
        raise NotImplementedError

    def _sort_full_items(self, user_id, *args, **kwargs):
        """Sort a list of items for a given user."""
        raise NotImplementedError

In [ ]:
from typing import Union, List, Optional
import torch
import torch.nn as nn


class MF(BaseModel, nn.Module):
    def __init__(self, config, n_users, n_items):
        BaseModel.__init__(self, config, n_users, n_items)
        nn.Module.__init__(self)
        self.config = config
        self.embedding_size = config["embedding_size"]
        self.n_users = n_users
        self.n_items = n_items
        torch.manual_seed(config['seed'])
        # define layers and loss
        self.user_embedding = nn.Embedding(self.n_users+1, self.embedding_size)
        self.item_embedding = nn.Embedding(self.n_items+1, self.embedding_size)
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)

    def forward(self, user, item):
        """Predicts the rating of a user for an item."""
        user_embed = self.user_embedding(user)
        item_embed = self.item_embedding(item)

        # Dot product between user and item embeddings to predict rating
        predicted_rating = (user_embed * item_embed).sum(1)

        return predicted_rating

    def get_full_sort_items(self, user, items):
        """Get a list of sorted items for a given user."""
        predicted_ratings = self.forward(user, items)
        sorted_items = self._sort_full_items(user, predicted_ratings, items)
        return sorted_items.tolist()

    def _sort_full_items(self, user, predicted_ratings, items):
        """Sort items based on their predicted ratings."""
        # Sort items based on ratings in descending order and return item indices
        _, sorted_indices = torch.sort(predicted_ratings, descending=True)
        return items[sorted_indices]

In [ ]:
import math
from typing import List, Tuple, Iterable, Optional
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class LightGCN(BaseModel, nn.Module):
    """
    LightGCN for recommendation.
    Reference: He et al., "LightGCN: Simplifying and Powering Graph Convolution Network for Recommendation"
    """

    def __init__(
        self,
        config,
        n_users: int,
        n_items: int,
        interactions: Iterable[Tuple[int, int]],
        n_layers: int = 3,
        embedding_dim: int = 64,
        device: Optional[torch.device] = None,
    ):
        """
        interactions: iterable of (user_id, item_id) pairs (duplicates okay, handled as multi-edges of weight 1)
        """
        BaseModel.__init__(self, config, n_users, n_items)
        nn.Module.__init__(self)

        self.n_users = n_users
        self.n_items = n_items
        self.n_layers = n_layers
        self.embedding_dim = embedding_dim
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        torch.manual_seed(config.get("seed", 2023))

        # Embeddings
        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        self.item_embedding = nn.Embedding(n_items, embedding_dim)
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)

        # Build normalized adjacency once
        self.Graph = self._build_normalized_adj(interactions).to(self.device)

        self.to(self.device)

    # ---------- Graph utilities ----------
    def _build_normalized_adj(self, interactions: Iterable[Tuple[int, int]]) -> torch.sparse.FloatTensor:
        """
        Build the symmetrically normalized adjacency matrix:
            A_hat = D^{-1/2} * A * D^{-1/2}
        for the bipartite user-item graph where nodes = users + items
        Size: (n_nodes, n_nodes) with n_nodes = n_users + n_items
        """
        n_nodes = self.n_users + self.n_items

        # Collect COO edges (user <-> item, undirected)
        rows = []
        cols = []
        vals = []

        for u, i, *rest in interactions:
            # Allow (u,i) or (u,i,rating) — rating ignored here (implicit 1)
            if u < 0 or u >= self.n_users or i < 0 or i >= self.n_items:
                continue
            i_offset = self.n_users + i  # item node index in unified graph
            # u -> i
            rows.append(u); cols.append(i_offset); vals.append(1.0)
            # i -> u
            rows.append(i_offset); cols.append(u); vals.append(1.0)

        if len(rows) == 0:
            # Empty graph fallback (identity to avoid NaNs)
            indices = torch.arange(n_nodes, dtype=torch.long)
            indices = torch.stack([indices, indices], dim=0)
            values = torch.ones(n_nodes, dtype=torch.float32)
            return torch.sparse_coo_tensor(indices, values, (n_nodes, n_nodes)).coalesce()

        indices = torch.tensor([rows, cols], dtype=torch.long)
        values = torch.tensor(vals, dtype=torch.float32)
        A = torch.sparse_coo_tensor(indices, values, (n_nodes, n_nodes)).coalesce()

        # Degree vector d = sum of rows
        deg = torch.sparse.sum(A, dim=1).to_dense()  # (n_nodes,)
        # Avoid divide-by-zero
        deg = torch.clamp(deg, min=1e-12)
        d_inv_sqrt = torch.pow(deg, -0.5)

        # Normalize values: for each edge (i,j), val *= d^-1/2[i] * d^-1/2[j]
        row, col = A.indices()
        norm_vals = A.values() * d_inv_sqrt[row] * d_inv_sqrt[col]

        A_hat = torch.sparse_coo_tensor(A.indices(), norm_vals, A.size()).coalesce()
        return A_hat

    # ---------- Embedding propagation ----------
    def propagate(self) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Perform K-layer LightGCN propagation and return final user & item embeddings.
        E^(0) = concat([U, I])  -> shape (n_users + n_items, d)
        E_final = 1/(K+1) * sum_{k=0..K} E^(k)
        """
        E_u0 = self.user_embedding.weight
        E_i0 = self.item_embedding.weight
        E0 = torch.cat([E_u0, E_i0], dim=0)  # (n_users + n_items, d)

        all_layers = [E0]
        x = E0
        for _ in range(self.n_layers):
            x = torch.sparse.mm(self.Graph, x)  # LightGCN propagation (no weights, no nonlinearity)
            all_layers.append(x)

        E = torch.stack(all_layers, dim=0).mean(dim=0)  # layer-wise average
        Eu, Ei = torch.split(E, [self.n_users, self.n_items], dim=0)
        return Eu, Ei

    # ---------- Scoring ----------
    def forward(self, users: torch.Tensor, items: torch.Tensor) -> torch.Tensor:
        """
        Predict scores for (users, items). Supports:
          - users: (B,), items: (B,)  -> elementwise scores
          - users: (B,), items: (N,)  -> broadcast users for all N items (returns (B,N))
        """
        users = users.to(self.device)
        items = items.to(self.device)

        Eu, Ei = self.propagate()

        if users.dim() == 1 and items.dim() == 1 and users.shape[0] == items.shape[0]:
            u_emb = Eu[users]                   # (B, d)
            i_emb = Ei[items]                   # (B, d)
            return (u_emb * i_emb).sum(dim=1)   # (B,)

        # Broadcast to (B, N)
        u_emb = Eu[users]                       # (B, d)
        i_emb = Ei[items]                       # (N, d)
        scores = u_emb @ i_emb.t()              # (B, N)
        return scores

    def predict(self, user_ids: List[int], item_ids: Optional[List[int]] = None) -> np.ndarray:
        """
        Convenience method to get scores as numpy.
        - If item_ids is None: scores for all items for each user -> shape (B, n_items)
        - Else: scores for user_ids x item_ids -> (B, len(item_ids))
        """
        self.eval()
        with torch.no_grad():
            u = torch.tensor(user_ids, dtype=torch.long, device=self.device)
            if item_ids is None:
                items = torch.arange(self.n_items, dtype=torch.long, device=self.device)
                scores = self.forward(u, items)        # (B, n_items)
            else:
                items = torch.tensor(item_ids, dtype=torch.long, device=self.device)
                scores = self.forward(u, items)        # (B, len(item_ids))
        return scores.detach().cpu().numpy()

    # ---------- Ranking API (BaseModel) ----------
    def get_full_sort_items(self, user_id: int, seen_items: Optional[Iterable[int]] = None, top_k: Optional[int] = None) -> List[int]:
        """
        Rank all items for a given user (descending by predicted score).
        Optionally drop previously seen items.
        """
        scores = self.predict([user_id], None).ravel()  # (n_items,)
        if seen_items is not None:
            # push seen items to bottom
            seen_items = [i for i in seen_items if 0 <= i < self.n_items]
            scores[np.array(seen_items, dtype=np.int64)] = -np.inf

        order = np.argsort(-scores)  # descending
        if top_k is not None:
            order = order[:top_k]
        return order.tolist()

    def _sort_full_items(self, user_id: int, predicted_ratings: torch.Tensor, items: torch.Tensor):
        # Not used here; keeping for BaseModel compatibility
        _, idx = torch.sort(predicted_ratings, descending=True)
        return items[idx]


class BPRLoss(nn.Module):
    """
    Pairwise Bayesian Personalized Ranking loss with L2 regularization on embeddings.
    """
    def __init__(self, reg: float = 1e-4):
        super().__init__()
        self.reg = reg

    def forward(self, u_emb, pos_emb, neg_emb):
        pos_scores = (u_emb * pos_emb).sum(dim=1)
        neg_scores = (u_emb * neg_emb).sum(dim=1)
        loss = -F.logsigmoid(pos_scores - neg_scores).mean()
        reg = (u_emb.norm(2).pow(2) + pos_emb.norm(2).pow(2) + neg_emb.norm(2).pow(2)) / u_emb.shape[0]
        return loss + self.reg * reg

In [ ]:
import os
from typing import Optional, Tuple, Iterable, List, Dict
import math

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset


class InteractionDataset(Dataset):
    """
    Dataset that yields dense user vectors (torch.float32).
    Accepts a scipy.sparse CSR matrix or a numpy array of shape (n_users, n_items).
    """
    def __init__(self, user_item_matrix):
        # Accept CSR matrix or numpy array
        if hasattr(user_item_matrix, "toarray") and hasattr(user_item_matrix, "tocsr"):
            self.mat = user_item_matrix.tocsr()
            self.is_sparse = True
            self.n_users, self.n_items = self.mat.shape
        else:
            self.mat = np.asarray(user_item_matrix, dtype=np.float32)
            self.is_sparse = False
            self.n_users, self.n_items = self.mat.shape

    def __len__(self):
        return self.n_users

    def __getitem__(self, idx):
        if self.is_sparse:
            row = self.mat.getrow(idx).toarray().astype(np.float32).squeeze(0)
            return torch.from_numpy(row)
        else:
            return torch.from_numpy(self.mat[idx].astype(np.float32))


class MultiVAE(BaseModel, nn.Module):
    """
    MultiVAE model (variational autoencoder for collaborative filtering).
    - encoder: two-layer MLP -> mu, logvar
    - decoder: linear mapping from latent z to item logits
    """
    def __init__(
        self,
        config: dict,
        n_users: int,
        n_items: int,
        hidden_dim: int = 600,
        latent_dim: int = 200,
        dropout=0.5,
        device: Optional[torch.device] = None,
    ):
        BaseModel.__init__(self, config, n_users, n_items)
        nn.Module.__init__(self)

        self.device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        self.n_users = n_users
        self.n_items = n_items

        # model dims
        self.hidden_dim = config.get("vae_hidden_dim", hidden_dim)
        self.latent_dim = config.get("vae_latent_dim", latent_dim)
        self.dropout = config.get("vae_dropout", dropout)

        # encoder
        self.encoder_fc1 = nn.Linear(n_items, self.hidden_dim)
        self.encoder_fc2 = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.mu_layer = nn.Linear(self.hidden_dim, self.latent_dim)
        self.logvar_layer = nn.Linear(self.hidden_dim, self.latent_dim)

        # decoder (linear + bias to produce logits over items)
        self.decoder = nn.Linear(self.latent_dim, n_items, bias=True)

        self.act = nn.Tanh()
        self.drop = nn.Dropout(self.dropout)

        # initialization
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)

        self.to(self.device)

    # ---------- VAE ops ----------
    def encode(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        x: (B, n_items) input user vectors
        returns mu, logvar (each (B, latent_dim))
        """
        h = self.drop(self.act(self.encoder_fc1(x)))
        h = self.drop(self.act(self.encoder_fc2(h)))
        mu = self.mu_layer(h)
        logvar = self.logvar_layer(h)
        return mu, logvar

    def reparameterize(self, mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z: torch.Tensor) -> torch.Tensor:
        """
        Returns logits for items: (B, n_items)
        """
        logits = self.decoder(z)
        return logits

    def forward(self, x: torch.Tensor, sample: bool = True) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        One forward pass:
        - encodes x -> mu, logvar
        - draws z (reparameterization)
        - decodes logits
        Returns: logits, mu, logvar
        """
        mu, logvar = self.encode(x)
        if self.training and sample:
            z = self.reparameterize(mu, logvar)
        else:
            z = mu  # use mean for deterministic inference
        logits = self.decode(z)
        return logits, mu, logvar

    # ---------- Loss / ELBO ----------
    def loss_function(
        self,
        logits: torch.Tensor,
        input_batch: torch.Tensor,
        mu: torch.Tensor,
        logvar: torch.Tensor,
        anneal: float = 1.0,
    ) -> Tuple[torch.Tensor, float, float]:
        """
        Multinomial likelihood version used in MultiVAE:
        recon_loss = - sum_j x_j * log_softmax(logits)_j

        KL = -0.5 * sum(1 + logvar - mu^2 - exp(logvar))
        total_loss = recon_loss + anneal * KL
        Returns (loss, recon_loss_scalar, kl_scalar)
        """
        # reconstruction: log softmax and weighted by counts
        log_softmax = F.log_softmax(logits, dim=1)  # (B, n_items)
        # input_batch may be counts (0/1 or counts). multiply and sum per sample
        recon = -torch.sum(log_softmax * input_batch, dim=1)  # (B,)
        recon_loss = torch.mean(recon)

        # KL
        kl = -0.5 * torch.sum(1.0 + logvar - mu.pow(2) - logvar.exp(), dim=1)  # (B,)
        kl_loss = torch.mean(kl)

        loss = recon_loss + anneal * kl_loss
        return loss, recon_loss.item(), kl_loss.item()


    # ---------- Embeddings helper (for ranking similar to LightGCN propagate) ----------
    def get_user_latent(self, x: torch.Tensor) -> torch.Tensor:
        """
        Return deterministic latent mu for x (no sampling).
        x: (B, n_items)
        returns mu: (B, latent_dim)
        """
        mu, logvar = self.encode(x)
        return mu

    # ---------- Prediction / Ranking (BaseModel methods) ----------
    def predict(self, user_vectors: torch.Tensor) -> np.ndarray:
        """
        Predict item scores for a batch of user vectors.
        user_vectors: torch.Tensor (B, n_items) on cpu or device
        returns: numpy array (B, n_items) of logits (higher = more recommended)
        """
        self.eval()
        with torch.no_grad():
            user_vectors = user_vectors.to(self.device)
            logits, mu, logvar = self.forward(user_vectors, sample=False)
            # Return logits (no softmax) so they can be sorted. move to cpu
            return logits.detach().cpu().numpy()

    def get_full_sort_items(self, user_id: int, user_item_matrix, seen_items: Optional[Iterable[int]] = None, top_k: Optional[int] = None) -> List[int]:
        """
        Rank all items for a given user.
        - user_item_matrix: the full user-item matrix (csr or numpy) to extract the user's vector
        - seen_items: optional iterable to mask (push to bottom)
        """
        # build user vector
        if hasattr(user_item_matrix, "getrow"):
            vec = user_item_matrix.getrow(user_id).toarray().astype(np.float32)
            user_vec = torch.from_numpy(vec).to(self.device)
        else:
            user_vec = torch.from_numpy(np.asarray(user_item_matrix[user_id], dtype=np.float32)).to(self.device)

        scores = self.predict(user_vec.unsqueeze(0)).ravel()  # (n_items,)

        if seen_items is not None:
            seen = [i for i in seen_items if 0 <= i < self.n_items]
            scores[np.array(seen, dtype=np.int64)] = -np.inf

        order = np.argsort(-scores)
        if top_k is not None:
            order = order[:top_k]
        return order.tolist()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


class FMModel(BaseModel, nn.Module):
    def __init__(self, config, n_users, n_items, n_factors=16):
        BaseModel.__init__(self, config, n_users, n_items)
        nn.Module.__init__(self)
        self.n_users = n_users
        self.n_items = n_items
        self.n_factors = n_factors

        # Linear terms
        self.user_bias = nn.Embedding(n_users, 1)
        self.item_bias = nn.Embedding(n_items, 1)

        # Factorization embeddings
        self.user_embedding = nn.Embedding(n_users, n_factors)
        self.item_embedding = nn.Embedding(n_items, n_factors)

        # Global bias
        self.global_bias = nn.Parameter(torch.zeros(1))

        # Init
        nn.init.normal_(self.user_embedding.weight, std=0.01)
        nn.init.normal_(self.item_embedding.weight, std=0.01)
        nn.init.zeros_(self.user_bias.weight)
        nn.init.zeros_(self.item_bias.weight)

    def forward(self, user_ids, item_ids):
        user_vec = self.user_embedding(user_ids)
        item_vec = self.item_embedding(item_ids)

        linear_terms = self.user_bias(user_ids) + self.item_bias(item_ids)
        interaction = torch.sum(user_vec * item_vec, dim=1, keepdim=True)

        score = self.global_bias + linear_terms + interaction
        return score.view(-1)

    def get_full_sort_items(self, user_id):
        device = next(self.parameters()).device
        user_id_tensor = torch.tensor([user_id], device=device)
        item_ids = torch.arange(self.n_items, device=device)
        user_id_expand = user_id_tensor.expand(self.n_items)
        scores = self.forward(user_id_expand, item_ids)
        return torch.argsort(scores, descending=True).tolist()

    def _sort_full_items(self, user_id):
        return self.get_full_sort_items(user_id)

In [ ]:
from __future__ import annotations
import math
import random
from typing import List, Dict, Tuple, Iterable

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

# ------------------------------
# Utility: set all seeds
# ------------------------------
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def sasrec_pointwise_step(model, batch, device, logit_clip=20.0):
    users, seqs, pos_items, neg_items, mask = batch
    seqs, pos_items, neg_items = seqs.to(device), pos_items.to(device), neg_items.to(device)
    mask = mask.to(device).bool()

    seq_out = model(seqs)
    item_emb = model.item_embedding.weight

    pos_logits = torch.sum(seq_out * item_emb[pos_items], dim=-1).clamp(-logit_clip, logit_clip)
    neg_logits = torch.sum(seq_out * item_emb[neg_items], dim=-1).clamp(-logit_clip, logit_clip)

    valid_mask = (pos_items > 0) & mask
    if valid_mask.sum() == 0:
        return torch.tensor(0.0, device=device, requires_grad=True)

    loss_pos = F.binary_cross_entropy_with_logits(pos_logits[valid_mask], torch.ones_like(pos_logits[valid_mask]))
    loss_neg = F.binary_cross_entropy_with_logits(neg_logits[valid_mask], torch.zeros_like(neg_logits[valid_mask]))

    loss = loss_pos + loss_neg

    # L2 reg, skip padding embedding
    if model.l2_emb > 0:
        loss += model.l2_emb * (item_emb[1:].norm(p=2) ** 2) / 2

    return loss


# Dataset & Collate
class SASRecDataset(Dataset):
    """Builds sequences for SASRec training.

    Args:
        user2items: dict mapping user_id -> list of interacted item ids (in time order)
        n_items: total number of items (max ID)
        max_seq_len: truncate/pad sequences to this length
        min_seq_len: smallest effective length (>=2 to have a next item)
    """
    def __init__(self,
                 user2items: Dict[int, List[int]],
                 n_items: int,
                 max_seq_len: int = 50,
                 min_seq_len: int = 2):
        self.user2items = user2items
        self.users = [u for u, seq in user2items.items() if len(seq) >= min_seq_len]
        self.n_items = n_items
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx: int):
        user = self.users[idx]
        full = self.user2items[user]
        # Truncate to latest max_seq_len
        seq = full[-self.max_seq_len:]
        return user, seq


def _pad_sequence(seq: list, max_len: int) -> list:
    """Left-pad sequence with 0 to max_len."""
    seq = seq[-max_len:]
    return [0] * (max_len - len(seq)) + seq

def _build_pos_items(seq: list) -> list:
    """Next-item targets, last position padded with 0."""
    return seq[1:] + [0]


def _sample_negatives(seq: list, n_items: int) -> list:
    """Sample negatives for each position, avoiding seq items."""
    user_set = set(seq)
    negatives = []
    for _ in range(len(seq)):
        neg = random.randint(1, n_items - 1)  # avoid 0
        while neg in user_set:
            neg = random.randint(1, n_items - 1)
        negatives.append(neg)
    return negatives


def sasrec_collate(batch, n_items: int, max_seq_len: int):
    users, seqs = zip(*batch)
    seqs = [_pad_sequence(s, max_seq_len) for s in seqs]

    pos_items = [_build_pos_items(s) for s in seqs]
    neg_items = [_sample_negatives(s, n_items) for s in seqs]
    mask = [[1 if x != 0 else 0 for x in s] for s in seqs]

    return (
        torch.tensor(users, dtype=torch.long),
        torch.tensor(seqs, dtype=torch.long),
        torch.tensor(pos_items, dtype=torch.long),
        torch.tensor(neg_items, dtype=torch.long),
        torch.tensor(mask, dtype=torch.float),
    )


def build_user2items(train_data):
    user2items = defaultdict(list)
    for u, i, r in sorted(train_data, key=lambda x: (x[0], x[2])):
        user2items[u].append(i)
    return user2items


class SASRec(nn.Module, BaseModel):
    def __init__(self, config, n_users: int, n_items: int):
        nn.Module.__init__(self)  # initialize nn.Module
        BaseModel.__init__(self, config, n_users, n_items)  # keep BaseModel logic

        self.n_items = n_items
        self.hidden_units = int(config.get("hidden_units", 128))
        self.max_seq_len = int(config.get("max_seq_len", 50))
        self.num_heads = int(config.get("num_heads", 2))
        self.num_blocks = int(config.get("num_blocks", 2))
        self.dropout_rate = float(config.get("dropout_rate", 0.2))
        self.l2_emb = float(config.get("l2_emb", 0.0))

        # Embeddings
        self.item_embedding = nn.Embedding(n_items + 1, self.hidden_units, padding_idx=0)
        self.position_embedding = nn.Embedding(self.max_seq_len, self.hidden_units)

        # Transformer blocks
        self.blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=self.hidden_units,
                nhead=self.num_heads,
                dim_feedforward=self.hidden_units * 4,
                dropout=self.dropout_rate,
                activation="gelu",
                batch_first=True,
            ) for _ in range(self.num_blocks)
        ])
        self.dropout = nn.Dropout(self.dropout_rate)
        self.layer_norm = nn.LayerNorm(self.hidden_units)

        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.item_embedding.weight, std=0.02)
        nn.init.normal_(self.position_embedding.weight, std=0.02)

    def _causal_mask(self, L: int, device=None):
        # True means masked in PyTorch
        mask = torch.triu(torch.ones(L, L, device=device), diagonal=1).bool()
        return mask

    def forward(self, item_seq: torch.Tensor) -> torch.Tensor:
        """
        Encode sequence safely, with padding & causal masks.
        Args:
            item_seq: (B, L) padded with 0 on left
        Returns:
            seq_out: (B, L, H)
        """
        B, L = item_seq.shape
        device = item_seq.device
        pos_ids = torch.arange(L, device=device).unsqueeze(0).expand(B, L)

        # Embeddings
        x = self.item_embedding(item_seq) + self.position_embedding(pos_ids)
        x = self.layer_norm(x)
        x = self.dropout(x)

        key_padding_mask = (item_seq == 0)  # True = pad
        attn_mask = self._causal_mask(L, device=device)  # True = masked

        for blk in self.blocks:
            x = blk(
            x,
            src_mask=attn_mask,
            src_key_padding_mask=key_padding_mask
            )
            # Safety: replace NaN/inf
            if torch.isnan(x).any() or torch.isinf(x).any():
               x = torch.nan_to_num(x, nan=0.0, posinf=1e4, neginf=-1e4)

        return x

    @torch.no_grad()
    def get_full_sort_items(self, user_id, user_seq: torch.Tensor):
        """Return sorted item IDs by score for a single user.
        Args:
            user_seq: (1, L)
        Returns:
            torch.Tensor of sorted item ids (desc)
        """
        self.eval()
        seq_out = self.forward(user_seq)[:, -1, :]  # (1, H)
        all_item_emb = self.item_embedding.weight  # (n_items+1, H)
        scores = torch.matmul(seq_out, all_item_emb.t()).squeeze(0)  # (n_items+1,)
        # Avoid recommending padding id 0
        scores[0] = -1e9
        return torch.argsort(scores, descending=True)

    def _sort_full_items(self, user_id, *args, **kwargs):
        raise NotImplementedError("Use get_full_sort_items(user_id, user_seq)")

In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import random
from collections import Counter
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr
from torch.utils.data import DataLoader


class Recommender:
    """
    Recommender System class
    """

    def __init__(self, config, logger, data):
        self.data = data
        self.config = config
        self.logger = logger
        self.page_size = config["page_size"]
        self.items_per_page = config["items_per_page"]
        self.random_k = config["rec_random_k"]
        self.train_data = []
        self.n_layers = 3
        self.embedding_dim = 4
        if config["rec_model"] == "MF":
           self.model = MF(config, self.data.get_user_num(), self.data.get_item_num())
        elif config["rec_model"] == "LightGCN":
           self.model = LightGCN(config, self.data.get_user_num(), self.data.get_item_num(), self.train_data, n_layers=self.n_layers, embedding_dim=self.embedding_dim)
        elif config["rec_model"] == "SASRec":
           self.model = SASRec(config, self.data.get_user_num(), self.data.get_item_num())
        elif config["rec_model"] == "MultiVAE":
           self.model = MultiVAE(config, self.data.get_user_num(), self.data.get_item_num())
        elif config["rec_model"] == "FM":
           self.model = FMModel(config, self.data.get_user_num(), self.data.get_item_num())
        else:
           raise ValueError(f"Unknown model: {config['rec_model']}")

        self.criterion = nn.MSELoss()
        if config["rec_model"] != "FM":
           self.optimizer = optim.Adam(self.model.parameters(), lr=config["lr"], weight_decay=1e-5)

        self.epoch_num = config["epoch_num"]
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.record = {}
        self.round_record = {}
        self.positive = {}
        self.interaction_dict = {}
        self.inter_df = None
        self.inter_num = 0
        for user in self.data.get_full_users():
            self.record[user] = []
            self.positive[user] = []
            self.round_record[user] = []
        self.user_data = {
            "user": [],
            "N_expose": [],
            "N_view": [],
            "N_like": [],
            "N_exit": [],
            "S_sat": []
            }
        self.rating_feeling = {
            "User": [],
            "Rating": [],
            "Feelings": []
        }

    def sample_bpr_triples(self,
                           user_pos_items: List[List[int]],
                           n_items: int,
                           batch_size: int,
                           device: torch.device,
                           ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        user_pos_items: list of lists; for each user u, a list of items they've interacted with (positive)
        returns (users, pos_items, neg_items) tensors of length batch_size
        """
        users = []
        pos = []
        neg = []
        for _ in range(batch_size):
           # sample a user with at least one positive
           while True:
               u = np.random.randint(0, len(user_pos_items))
               if len(user_pos_items[u]) > 0:
                  break

           i = np.random.choice(user_pos_items[u])
           # sample a negative item

           while True:
               j = np.random.randint(0, n_items)
               if j not in user_pos_items[u]:
                  break

           users.append(u)
           pos.append(i)
           neg.append(j)

        return (
        torch.tensor(users, dtype=torch.long, device=self.device),
        torch.tensor(pos, dtype=torch.long, device=self.device),
        torch.tensor(neg, dtype=torch.long, device=self.device),
        )

    def train_lightgcn_bpr(self,
        reg: float = 1e-4,
        log: bool = True):
        """
        Train LightGCN with BPR loss.
        - train_interactions/val_interactions can be (u,i) or (u,i,rating>0) tuples.
        - If ckpt_path is provided, saves the best (by simple val recall proxy) state dict.
        """

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config["lr"])
        self.criterion = BPRLoss(reg=reg)

        # Build user -> positives list
        user_pos = [[] for _ in range(self.model.n_users)]
        for u, i, *rest in self.train_data:
            if 0 <= u < self.model.n_users and 0 <= i < self.model.n_items:
               user_pos[u].append(i)

        # Basic validation proxy: count of positives ranked in top-10 (cheap & optional)
        def quick_val_topk_hits(k: int = 10) -> float:
            if self.val_data is None:
               return -1.0
            # build val positives per user
            val_pos = [[] for _ in range(self.model.n_users)]
            for u, i, *rest in self.val_data:
               if 0 <= u < self.model.n_users and 0 <= i < self.model.n_items:
                   val_pos[u].append(i)

            hits = 0
            total = 0

            for u in range(self.model.n_users):
               if not val_pos[u]:
                  continue
               recs = self.model.get_full_sort_items(u, seen_items=set(user_pos[u]), top_k=k)
               s = set(val_pos[u])
               hits += len([r for r in recs if r in s])
               total += min(k, len(s))
            return hits / total if total > 0 else -1.0

        best_metric = -math.inf

        # Build full checkpoint file path
        ckpt_file = os.path.join(self.config['checkpoint_path'], "best_lightGCN_model.pth")
        os.makedirs(self.config['checkpoint_path'], exist_ok=True)

        for epoch in range(1, self.epoch_num + 1):
            self.model.train()

            # One epoch of mini-batch BPR
            n_steps = max(1, sum(len(v) for v in user_pos) // max(1, config['batch_size']))
            losses = []
            for _ in range(n_steps):
                users, pos_items, neg_items = self.sample_bpr_triples(user_pos, self.model.n_items, config['batch_size'], self.device)
                Eu, Ei = self.model.propagate()
                u_emb = Eu[users]
                p_emb = Ei[pos_items]
                n_emb = Ei[neg_items]
                loss = self.criterion(u_emb, p_emb, n_emb)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                losses.append(loss.item())

            # quick validation metric
            metric = quick_val_topk_hits(k=10)
            if log:
               print(f"[Epoch {epoch:3d}] BPR Loss: {np.mean(losses):.4f} | Val@10: {metric:.4f}")

            # Save checkpoint if validation improves
            if metric > best_metric:
               best_metric = metric
               torch.save({
                   "epoch": self.epoch_num + 1,
                   "model_state_dict": self.model.state_dict(),
                   "n_users": self.data.get_user_num(),
                   "n_items": self.data.get_item_num(),
                   "n_layers": self.n_layers,
                   "embedding_dim": self.embedding_dim,
                   "metric": metric,
                   }, ckpt_file)
               self.logger.info(f"Best model updated at epoch {epoch+1}, saved to {ckpt_file}")

        # Load best (optional)
        if ckpt_file is not None and best_metric > -math.inf:
           # At the end, reload the best weights for inference
           checkpoint = torch.load(ckpt_file)
           self.model.load_state_dict(checkpoint['model_state_dict'])

    def bce_sampled_loss(self, seq_out: torch.Tensor,
        pos_items: torch.Tensor,
        neg_items: torch.Tensor,
        item_embedding: nn.Embedding,
        mask: torch.Tensor,
        l2_emb: float = 0.0) -> torch.Tensor:
        """Binary cross-entropy on sampled positives/negatives per position.
        Args:
            seq_out: (B, L, H)
            pos_items: (B, L) next-item ids (0 where no target)
            neg_items: (B, L) sampled negatives (0 where no target)
            item_embedding: embedding module (to fetch item vectors)
            mask: (B, L) boolean, True where a target exists (i.e., pos_items > 0)
            l2_emb: weight decay on item embeddings (regularizes pos/neg lookups)
        """
        B, L, H = seq_out.shape

        pos_vecs = item_embedding(pos_items)  # (B, L, H)
        neg_vecs = item_embedding(neg_items)  # (B, L, H)

        pos_logits = (seq_out * pos_vecs).sum(-1)  # (B, L)
        neg_logits = (seq_out * neg_vecs).sum(-1)  # (B, L)

        # Targets: pos -> 1, neg -> 0
        pos_loss = F.binary_cross_entropy_with_logits(pos_logits[mask], torch.ones_like(pos_logits[mask]))
        neg_loss = F.binary_cross_entropy_with_logits(neg_logits[mask], torch.zeros_like(neg_logits[mask]))
        loss = pos_loss + neg_loss

        if l2_emb > 0:
           reg = (pos_vecs[mask].pow(2).sum() + neg_vecs[mask].pow(2).sum()) / mask.sum().clamp_min(1)
           loss = loss + l2_emb * reg
        return loss

    def train_sasrec(self, grad_clip=1.0, logit_clip=20.0):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # Build user->items dict and dataset
        user2items = build_user2items(self.train_data)
        n_items_global = int(self.data.get_item_num())
        max_seq_len = self.model.max_seq_len

        train_dataset = SASRecDataset(user2items, n_items=n_items_global, max_seq_len=max_seq_len)
        train_loader = DataLoader(
            train_dataset,
            batch_size=self.config['batch_size'],
            shuffle=True,
            collate_fn=lambda batch: sasrec_collate(batch, n_items=n_items_global, max_seq_len=max_seq_len)
            )

        # Validation loader
        val_user2items = build_user2items(self.val_data)
        val_dataset = SASRecDataset(val_user2items, n_items=n_items_global, max_seq_len=max_seq_len)
        val_loader = DataLoader(
            val_dataset,
            batch_size=self.config['batch_size'],
            shuffle=False,
            collate_fn=lambda batch: sasrec_collate(batch, n_items=n_items_global, max_seq_len=max_seq_len)
            )

        # Checkpoint setup
        ckpt_file = os.path.join(self.config['checkpoint_path'], "best_SASRec_model.pth")
        os.makedirs(self.config['checkpoint_path'], exist_ok=True)
        best_metric = -float("inf")

        for epoch in range(1, self.epoch_num + 1):
           self.model.train()
           running = 0.0
           n_steps = 0

           for users, seqs, pos, neg, umask in train_loader:
              users, seqs, pos, neg, umask = (
                users.to(self.device),
                seqs.to(self.device),
                pos.to(self.device),
                neg.to(self.device),
                umask.to(self.device).bool(),  # ensure bool type
                )

              # Compute mask dynamically
              mask = pos > 0
              mask = mask.bool()
              if mask.sum() == 0:
                 continue  # skip batch with no valid positions

              # Compute stable loss
              loss = sasrec_pointwise_step(self.model, (users, seqs, pos, neg, mask), device=self.device, logit_clip=logit_clip)

              self.optimizer.zero_grad(set_to_none=True)
              loss.backward()
              if grad_clip is not None:
                 nn.utils.clip_grad_norm_(self.model.parameters(), grad_clip)
              self.optimizer.step()

              running += loss.item()
              n_steps += 1

           avg_loss = running / max(1, n_steps)
           print(f"Epoch {epoch}/{self.epoch_num} - train loss: {avg_loss:.4f}")

           # Validation
           if val_loader is not None:
              self.model.eval()
              val_loss = 0.0
              n_val_steps = 0
              with torch.no_grad():
                  for users, seqs, pos, neg, umask in val_loader:
                     users, seqs, pos, neg, umask = (
                         users.to(self.device),
                         seqs.to(self.device),
                         pos.to(self.device),
                         neg.to(self.device),
                         umask.to(self.device).bool(),)

                     mask = pos > 0
                     if mask.sum() == 0:
                        # print("Skipped empty batch")
                        continue

                     loss = sasrec_pointwise_step(self.model, (users, seqs, pos, neg, mask), device=self.device, logit_clip=logit_clip)
                    #  print("Batch loss:", loss.item())
                     val_loss += loss.item()
                     n_val_steps += 1
              val_loss /= max(1, n_val_steps)
              metric = -val_loss
              print(f"Epoch {epoch}, Val Loss: {val_loss:.4f}")
           else:
              metric = -avg_loss

           # Save best model
           if metric > best_metric:
              best_metric = metric
              torch.save({
                "epoch": epoch,
                "model_state_dict": self.model.state_dict(),
                "metric": metric,
              }, ckpt_file)
              print(f"Best model updated at epoch {epoch}, saved to {ckpt_file}")

        # Load best model after training
        checkpoint = torch.load(ckpt_file, map_location=self.device)
        self.model.load_state_dict(checkpoint["model_state_dict"])


    def train_fm(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.config['lr'])
        self.criterion = nn.MSELoss()

        best_metric = -math.inf
        ckpt_file = os.path.join(self.config['checkpoint_path'], "best_FM_model.pth")
        os.makedirs(self.config['checkpoint_path'], exist_ok=True)

        # 🔹 Wrap datasets into DataLoaders
        train_loader = DataLoader(self.train_data, batch_size=self.config['batch_size'], shuffle=True)
        val_loader = None
        if self.val_data is not None:
           val_loader = DataLoader(self.val_data, batch_size=self.config['batch_size'], shuffle=False)


        for epoch in range(self.epoch_num):
           self.model.train()
           total_loss = 0

           for user, item, rating in train_loader:  # must be DataLoader
              user, item, rating = user.to(self.device), item.to(self.device), rating.float().to(self.device)

              self.optimizer.zero_grad()
              preds = self.model(user, item)
              loss = self.criterion(preds, rating)
              loss.backward()
              self.optimizer.step()

              total_loss += loss.item()

           avg_loss = total_loss / len(train_loader)
           print(f"Epoch {epoch+1}/{self.epoch_num}, Train Loss: {avg_loss:.4f}")

           metric = None
           if val_loader is not None:
              self.model.eval()
              with torch.no_grad():
                  val_loss = 0
                  for user, item, rating in val_loader:
                      user, item, rating = user.to(self.device), item.to(self.device), rating.to(self.device)
                      preds = self.model(user, item)
                      val_loss += self.criterion(preds, rating).item()
                  val_loss /= len(val_loader)

              metric = -val_loss
              print(f"Epoch {epoch+1}, Val Loss: {val_loss:.4f}")
           else:
              metric = -avg_loss  # fallback if no validation set

           # save best model
           if metric > best_metric:
              best_metric = metric
              torch.save({
                "epoch": epoch+1,
                "model_state_dict": self.model.state_dict(),
                "n_users": self.model.n_users,
                "n_items": self.model.n_items,
                "n_factors": self.model.n_factors,
                "metric": metric,
                }, ckpt_file)
              print(f"Best model updated at epoch {epoch+1}, saved to {ckpt_file}")

        checkpoint = torch.load(ckpt_file, map_location=self.device)
        self.model.load_state_dict(checkpoint["model_state_dict"])


    def train_multivae(self,
        weight_decay: float = 0.0,
        anneal_cap: float = 0.2,
        total_anneal_steps: int = 200000,
        patience: int = 100,
        verbose: bool = True,):
        """
        Train MultiVAE on user-item matrix (CSR or dense numpy).
        - anneal_cap: maximum beta for KL weighting
        - total_anneal_steps: number of optimization steps over which to ramp beta from 0->anneal_cap
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        dataset = InteractionDataset(self.train_matrix)
        loader = DataLoader(dataset, batch_size=config['batch_size'], shuffle=True, drop_last=False)

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config['lr'], weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.5)
        update_count = 0
        best_val_loss = float("inf")

        # Build full checkpoint file path
        ckpt_file = os.path.join(self.config['checkpoint_path'], "best_MultiVAE_model.pth")
        os.makedirs(self.config['checkpoint_path'], exist_ok=True)

        wait = 0

        # AMP scaler (for mixed precision training)
        scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

        # validation dataset (create once, not per epoch)
        val_loader, val_dataset = None, None
        if self.val_matrix is not None:
           val_dataset = InteractionDataset(self.val_matrix)
           val_loader = DataLoader(val_dataset, batch_size=self.config['batch_size'], shuffle=False)

        for epoch in range(1, self.epoch_num + 1):
            self.model.train()
            epoch_loss, epoch_recon, epoch_kl, n_batches = 0.0, 0.0, 0.0, 0

            for batch in loader:
                batch = batch.to(self.device).float()
                assert batch.shape[1] == self.model.n_items, "Batch dimension mismatch!"

                # anneal factor
                if total_anneal_steps > 0:
                   anneal = min(anneal_cap, update_count / total_anneal_steps)
                else:
                   anneal = anneal_cap

                self.optimizer.zero_grad()

                with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                    logits, mu, logvar = self.model(batch, sample=True)
                    # clamp logvar inside model (optional, numerical stability)
                    logvar = torch.clamp(logvar, min=-10, max=10)
                    loss, recon_l, kl_l = self.model.loss_function(logits, batch, mu, logvar, anneal)

                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=5.0)
                scaler.step(self.optimizer)
                scaler.update()

                epoch_loss += loss.item()
                epoch_recon += recon_l
                epoch_kl += kl_l
                update_count += 1
                n_batches += 1

            scheduler.step()
            avg_train_elbo = epoch_loss / n_batches

            if verbose:
               print(f"[Epoch {epoch}] Train ELBO: {avg_train_elbo:.4f} | "
                  f"Recon: {epoch_recon / n_batches:.4f} | KL: {epoch_kl / n_batches:.4f} | "
                  f"Anneal: {anneal:.4f}")

            # ---------- Validation ----------
            if val_loader is not None:
               self.model.eval()
               val_losses = []
               with torch.no_grad():
                   for vb in val_loader:
                      vb = vb.to(self.device).float()
                      logits, mu, logvar = self.model(vb, sample=False)
                      logvar = torch.clamp(logvar, min=-10, max=10)
                      vloss, vrec, vkl = self.model.loss_function(logits, vb, mu, logvar, anneal)
                      val_losses.append(vloss.item() * len(vb))

               val_loss = float(np.sum(val_losses) / len(val_dataset))

               if verbose:
                 print(f"  -> Val ELBO: {val_loss:.4f}")

               # save best with early stopping
               if val_loss < best_val_loss:
                  best_val_loss = val_loss
                  wait = 0
                  torch.save({
                    "epoch": epoch,
                    "model_state_dict": self.model.state_dict(),
                    "optimizer": self.optimizer.state_dict(),
                    "n_users": self.data.get_user_num(),
                    "n_items": self.data.get_item_num(),
                    "hidden_dim": self.config["vae_hidden_dim"],
                    "latent_dim": self.config["vae_latent_dim"],
                    "config": self.config,
                    }, ckpt_file)
                  self.logger.info(f"Best model updated at epoch {epoch}, saved to {ckpt_file}")
               else:
                  wait += 1
                  if wait >= patience:
                     print(f"Early stopping triggered at epoch {epoch}")
                     break

        # reload best model
        checkpoint = torch.load(ckpt_file, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])

    def save_model(self, path):
        torch.save(self.model.state_dict(), path)

    def load_model(self, path):
        self.model.load_state_dict(torch.load(path))

    def swap_items(self, lst, page_size, random_k):
        total_pages = len(lst) // page_size
        lst = lst[: total_pages * page_size]
        for page in range(1, total_pages // 2 + 1):
            start_idx = (page - 1) * page_size
            end_idx = start_idx + page_size - 1
            symmetric_start_idx = (total_pages - page) * page_size
            symmetric_end_idx = symmetric_start_idx + page_size

            for k in range(1, random_k + 1):
                lst[end_idx - k], lst[symmetric_end_idx - k] = (
                    lst[symmetric_end_idx - k],
                    lst[end_idx - k],
                )

        return lst

    def add_random_items(self, user, item_ids):
        item_ids = self.swap_items(item_ids, self.page_size, self.random_k)
        return item_ids

    def ordered_probit_loglik(self, y_true, y_pred_int, K=5, taus=None):
        """
        Compute log-likelihood for ordered probit model given integer predictions.

        y_true : list or array
           True ratings (1..K).
        y_pred_int : list or array
           Predicted integer ratings (1..K).
        K : int
           Number of rating categories (default 5).
        taus : list or array, optional
           Thresholds (default: equally spaced).
        """

        assert len(y_true) == len(y_pred_int), "Mismatch in true vs predicted length"
        ll = 0.0
        for t, p in zip(y_true, y_pred_int):
           probs = self.data.ordered_probit_probs(p, K, taus)
           ll += self.data.safe_log(probs[t-1])  # subtract 1 for 0-based index
        avg_ll = ll / len(y_true)
        return ll, avg_ll

    def update_user_interactions(self, user_id, new_items):
        """
        Updates the directory of user_id and interacted_items.
        - interaction_dict: dict mapping user_id -> set of interacted item ids
        - user_id: int or str
        - new_items: iterable of item ids (list, set, etc)

        After calling, interaction_dict[user_id] contains all unique interacted items.
        """
        # Ensure the user's interaction set exists
        if user_id not in self.interaction_dict:
          self.interaction_dict[user_id] = set()

        new_items = set(new_items) - self.interaction_dict[user_id]
        self.interaction_dict[user_id].update(new_items)

    def get_full_manual_items(self, user_id, gt_ratio, rd_ratio, total_items, read = None, heard = None):
        """
        Get a list of manual items for a given user.
        """
        gtruth_items = self.data.interrating[user_id]
        gt_items = [item for item, rating in gtruth_items]
        rd_items = self.data.get_full_items()

        # Remove any gt_items from rd_items to avoid duplicates
        rd_items = list(set(rd_items) - set(gt_items))

        # 1. Determine counts
        total_ratio = gt_ratio + rd_ratio
        gt_count = round(total_items * gt_ratio / total_ratio)
        rd_count = total_items - gt_count

        # Make sure we don't try to sample more than available
        gt_count = min(gt_count, len(gt_items))
        rd_count = min(rd_count, len(rd_items))

        # 2. Randomly sample
        chosen_gt = random.sample(gt_items, gt_count) if gt_count > 0 else []
        chosen_rd = random.sample(rd_items, rd_count) if rd_count > 0 else []

        # 3. Combine and shuffle if desired
        final_items = chosen_gt + chosen_rd

        # items discriptions
        sorted_item_names = self.data.get_item_names(final_items)
        description = self.data.get_item_description_by_id(final_items)
        eb_item = [
            sorted_item_names[i]
            + ";;"
            + description[i]
            + ";; Genre: "
            + self.data.get_genres_by_id([final_items[i]])[0]
            for i in range(len(sorted_item_names))
        ]
        return final_items, eb_item, chosen_gt

    def get_full_sort_items(self, user, random=False):
        """
        Get a list of sorted items for a given user.
        """
        items = self.data.get_full_items()
        user_tensor = torch.tensor(user)
        items_tensor = torch.tensor(items)
        sorted_items = self.model.get_full_sort_items(user_tensor, items_tensor)
        if self.random_k > 0 and random == True:
            sorted_items = self.add_random_items(user, sorted_items)
        sorted_items = [item for item in sorted_items if item not in self.record[user]]
        sorted_item_names = self.data.get_item_names(sorted_items)
        description = self.data.get_item_description_by_id(sorted_items)
        items = [
            sorted_item_names[i]
            + ";;"
            + description[i]
            + ";; Genre: "
            + self.data.get_genres_by_id([sorted_items[i]])[0]
            for i in range(len(sorted_item_names))
        ]
        return sorted_items, items

    def get_item(self, idx):
        item_name = self.data.get_item_names([idx])[0]
        description = self.data.get_item_description_by_id([idx])[0]
        item = item_name + ";;" + description
        return item

    def get_search_items(self, item_name):
        return self.data.search_items(item_name)

    def get_inter_num(self):
        return self.inter_num

    def update_history_by_name(self, user_id, item_names):
        """
        Update the history of a given user.
        """
        item_names = [item_name.strip(" <>'\"") for item_name in item_names]
        item_ids = self.data.get_item_ids(item_names)
        self.record[user_id].extend(item_ids)

    def update_history_by_id(self, user_id, item_ids):
        """
        Update the history of a given user.
        """
        self.record[user_id].extend(item_ids)

    def update_positive(self, user_id, item_names):
        """
        Update the positive history of a given user.
        """
        item_ids = self.data.get_item_ids(item_names)
        if len(item_ids) == 0:
            return
        self.positive[user_id].extend(item_ids)
        self.inter_num += len(item_ids)

    def update_positive_by_id(self, user_id, item_id):
        """
        Update the history of a given user.
        """
        self.positive[user_id].append(item_id)

    def save_interaction(self):
        """
        Save the interaction history to a csv file.
        """
        inters = []
        users = self.data.get_full_users()
        for user in users:
            for item in self.positive[user]:
                new_row = {"user_id": user, "item_id": item, "rating": 1}
                inters.append(new_row)

            for item in self.record[user]:
                if item in self.positive[user]:
                    continue
                new_row = {"user_id": user, "item_id": item, "rating": 0}
                inters.append(new_row)

        df = pd.DataFrame(inters)
        df.to_csv(
            self.config["interaction_path"],
            index=False,
        )

        self.inter_df = df

    def add_train_data(self, user, item, label):
        self.train_data.append((user, item, label))

    def clear_train_data(self):
        self.train_data = []

    def add_user(self, user_id, N_expose, N_view, N_like, N_exit, S_sat):
        self.user_data["user"].append(user_id)
        self.user_data["N_expose"].append(N_expose)
        self.user_data["N_view"].append(N_view)
        self.user_data["N_like"].append(N_like)
        self.user_data["N_exit"].append(N_exit)
        self.user_data["S_sat"].append(S_sat)

    def add_review(self, user_id, rating, feelings):
        self.rating_feeling["User"].append(user_id)
        self.rating_feeling["Rating"].append(rating)
        self.rating_feeling["Feelings"].append(feelings)

    def satisfaction_metrics(self):
        sm_df = pd.DataFrame(self.user_data)
        if len(sm_df) == 0:
           return None  # no data yet

        metrics = {}
        sm_df["view_ratio"] = sm_df["N_view"] / sm_df["N_expose"]
        sm_df["like_ratio"] = sm_df["N_like"] / sm_df["N_expose"]

        metrics["P_view"] = sm_df["view_ratio"].mean()
        metrics["N_like"] = sm_df["N_like"].mean()
        metrics["P_like"] = sm_df["like_ratio"].mean()
        metrics["N_exit"] = sm_df["N_exit"].mean()
        metrics["S_sat"] = sm_df["S_sat"].mean()

        return metrics

    def get_entropy(
        self,
    ):
        tot_entropy = 0
        for user in self.record.keys():
            inters = self.record[user]
            genres = self.data.get_genres_by_id(inters)
            entropy = calculate_entropy(genres)
            tot_entropy += entropy

        return tot_entropy / len(self.record.keys())

    def check_train_data(self):
        """
        Print or inspect the training data.
        """
        print("Training Data:")
        for user, item, label in self.train_data:
            print(f"User: {user}, Item: {item}, Label: {label}")

    def create_train_data(self):
        """
        Create a training dataset with random samples.

        Args:
            num_samples (int): Number of samples to generate.
        """
        self.clear_train_data()  # Clear existing training data
        all_data = self.data.interrating  # You need to implement this or use available interaction data

        # Convert dict to list of (user, item, label)
        # triplets = []
        for user, interactions in all_data.items():
            for item, rating in interactions:
                self.add_train_data(user, item, float(rating))
                # triplets.append((user, item, float(rating)))  # keep exact rating

        # Split 80% train, 20% temp (to further split into val/test)
        self.train_data, self.temp_data = train_test_split(self.train_data, test_size=0.2, random_state=2025)

        # Split temp into 10% val and 10% test (from the total dataset)
        self.val_data, self.test_data = train_test_split(self.temp_data, test_size=0.5, random_state=2025)

        train_users = max([u for u, i, r in self.train_data]) + 1
        train_items = max([i for u, i, r in self.train_data]) + 1

        val_users = max([u for u, i, r in self.val_data]) + 1
        val_items = max([i for u, i, r in self.val_data]) + 1

        test_users = max([u for u, i, r in self.test_data]) + 1
        test_items = max([i for u, i, r in self.test_data]) + 1

        n_items_global = int(self.data.get_item_num())

        # Initialize user-item matrix
        self.train_matrix = np.zeros((train_users, n_items_global), dtype=np.float32)
        self.val_matrix = np.zeros((val_users, n_items_global), dtype=np.float32)
        self.test_matrix = np.zeros((test_users, n_items_global), dtype=np.float32)

        # Fill interactions safely
        for u, i, r in self.train_data:
           if i >= n_items_global: continue  # skip bad indices
           self.train_matrix[u, i] = r

        for u, i, r in self.val_data:
           if i >= n_items_global: continue
           self.val_matrix[u, i] = r

        for u, i, r in self.test_data:
           if i >= n_items_global: continue
           self.test_matrix[u, i] = r



    def calculate_user_metrics(
        self, user_id, sim_recommended, all_items, threshold = 3):
        """
        Evaluate precision, recall, (optionally real) accuracy, and F1 for a single user.

        Returns:
            dict: { 'precision': float, 'recall': float, 'accuracy': float, 'f1': float }
        """

        if user_id not in self.data.interrating:
           return {'precision': 0, 'recall': 0, 'accuracy': 0, 'f1': 0}

        ground_truth_pairs = self.data.interrating[user_id]
        gt_relevant = set(item for item, rating in ground_truth_pairs if rating >= threshold and item in all_items)
        sim_recommended = set(sim_recommended)
        all_items = set(all_items)

        TP = len(gt_relevant & sim_recommended)
        FP = len(sim_recommended - gt_relevant)
        FN = len(gt_relevant - sim_recommended)
        TN = len(all_items - (gt_relevant | sim_recommended))

        precision = TP / (TP + FP) if (TP + FP) else 0.0
        recall = TP / (TP + FN) if (TP + FN) else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0.0
        accuracy = (TP + TN) / len(all_items) if all_items else 0

        print("precision:", precision, "recall:", recall, "accuracy:", accuracy, "f1:", f1)
        return precision, recall, accuracy, f1

    def precisionandrecallk(
        self, user_id, recommended, k):
        if user_id not in self.data.interrating:
           return {'precision_at_k': 0, 'recall_at_k': 0}

        sim_recommended = list(dict.fromkeys(recommended))
        ground_truth_pairs = self.data.interrating[user_id]
        gt_relevant = set(item for item, rating in ground_truth_pairs if rating >= 3)
        recommended_at_k = sim_recommended[:k]
        hits = sum([1 for item in recommended_at_k if item in gt_relevant])
        precision_at_k = hits / k
        recall_at_k = hits / len(gt_relevant) if gt_relevant else 0
        return precision_at_k, recall_at_k


    def calculation_of_rating(self, user_id, item_names, book_rating):
        item_ids = self.data.get_item_ids([item_names])
        if user_id in self.data.interrating:
           # Check for item in user's ratings
           for (itm, rating) in self.data.interrating[user_id]:
               if itm == item_ids[0]:
                  return (rating, book_rating)

        # If not found
        return (0, book_rating)


    def calc_mse_rmse_rating_percentages(self, rating_pairs):

        print("Incoming rating_pairs:", rating_pairs[:20])  # show first 20 pairs
        print("Total pairs:", len(rating_pairs))

        # Remove pairs with zero in ground truth or predicted rating
        filtered_pairs = [(gt, pred) for gt, pred in rating_pairs
                          if int(gt) != 0]

        print("After filtering:", filtered_pairs[:20])
        print("Remaining pairs:", len(filtered_pairs))

        if not filtered_pairs:
           # No valid data after filtering
           return None, None, {}, {}, None, None, None

        # Convert ratings to int
        gt = [int(gt) for gt, pred in filtered_pairs]
        pred = [int(pred) for gt, pred in filtered_pairs]
        mse = np.mean([(g - p) ** 2 for g, p in zip(gt, pred)])
        rmse = np.sqrt(mse)
        loglike, ob_loglike = self.ordered_probit_loglik(gt, pred)
        rho, p_value = spearmanr(gt, pred)

        gt_count = Counter(gt)
        pred_count = Counter(pred)
        total = len(filtered_pairs)

        gt_pct = {r: gt_count.get(r, 0) / total * 100 for r in range(1, 6)}
        pred_pct = {r: pred_count.get(r, 0) / total * 100 for r in range(1, 6)}
        return mse, rmse, gt_pct, pred_pct, loglike, ob_loglike, rho


    def test_recommendations(self, user_id):
        # Get the full list of items
        all_items = self.data.get_full_items()

        # Convert the user ID to tensor
        user_tensor = torch.tensor(user_id)

        # Convert all items to tensor
        items_tensor = torch.tensor(all_items)

        # Get sorted items based on the model's prediction
        sorted_items = self.model.get_full_sort_items(user_tensor, items_tensor)

        # Filter out items that are already in the user's history
        recommended_items = [item for item in sorted_items if item not in self.record[user_id]]

        # Return the recommended items
        return recommended_items

    def evaluate(self, dataset):
        self.model.eval()
        users = torch.tensor([x[0] for x in dataset])
        items = torch.tensor([x[1] for x in dataset])
        labels = torch.tensor([x[2] for x in dataset]).float()

        with torch.no_grad():
             outputs = self.model(users, items)
             loss = self.criterion(outputs, labels)
        return loss.item()

    def load_checkpoint(self, path="best_model.pth", resume_training=False):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint["model_state_dict"])
        if resume_training:
           # Load optimizer state to resume training exactly where it left off
           self.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
           start_epoch = checkpoint["epoch"]
           self.logger.info(f"Resuming training from epoch {start_epoch}")
           return start_epoch
        else:
           self.model.eval()  # set to eval mode for inference

    def train_mf(self):
        if len(self.train_data) == 0:
            print("No training data!")
            return

        users = [x[0] for x in self.train_data]
        items = [x[1] for x in self.train_data]
        labels = [x[2] for x in self.train_data]


        dataset = torch.utils.data.TensorDataset(
        torch.tensor(users), torch.tensor(items), torch.tensor(labels))

        train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=self.config["batch_size"], shuffle=True)

        self.model.train()

        best_val_loss = float("inf")

        # Build full checkpoint file path
        ckpt_file = os.path.join(self.config['checkpoint_path'], "best_MF_model.pth")
        os.makedirs(self.config['checkpoint_path'], exist_ok=True)

        for epoch in range(self.epoch_num):
            epoch_loss = 0.0

            for user, item, label in train_loader:

                self.optimizer.zero_grad()
                outputs = self.model(user, item)
                loss = self.criterion(outputs, label.float())
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()

            val_loss = self.evaluate(self.val_data)  # Evaluate on validation set

            self.logger.info(
            f"Epoch {epoch+1}/{self.epoch_num}, Train Loss: {epoch_loss/len(train_loader):.4f}, "
            f"Val Loss: {val_loss:.4f}")

            # Save checkpoint if validation improves
            if val_loss < best_val_loss:
               best_val_loss = val_loss
               torch.save({
                "epoch": epoch + 1,
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "val_loss": val_loss,
                }, ckpt_file)
               self.logger.info(f"Best model updated at epoch {epoch+1}, saved to {ckpt_file}")

        # At the end, reload the best weights for inference
        checkpoint = torch.load(ckpt_file)
        self.model.load_state_dict(checkpoint['model_state_dict'])

    def load_best_model(self):
        if self.config['rec_model'] == 'MF':
           ckpt_file = os.path.join(self.config['checkpoint_path'], "best_MF_model.pth")
        elif self.config['rec_model'] == 'MultiVAE':
           ckpt_file = os.path.join(self.config['checkpoint_path'], "best_MultiVAE_model.pth")
        elif self.config['rec_model'] == 'LightGCN':
           ckpt_file = os.path.join(self.config['checkpoint_path'], "best_lightGCN_model.pth")
        elif self.config['rec_model'] == 'FM':
           ckpt_file = os.path.join(self.config['checkpoint_path'], "best_FM_model.pth")
        elif self.config['rec_model'] == 'SASRec':
           ckpt_file = os.path.join(self.config['checkpoint_path'], "best_SASRec_model.pth")
        else:
           raise ValueError(f"Unknown model type: {self.config['rec_model']}")

        # Build full checkpoint file path
        checkpoint = torch.load(ckpt_file)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()
        self.logger.info(f"Loaded best model from {ckpt_file}")

    def get_rec_discription(self, final_items):
        # items discriptions
        sorted_item_names = self.data.get_item_names(final_items)
        description = self.data.get_item_description_by_id(final_items)
        eb_item = [
            sorted_item_names[i]
            + ";;"
            + description[i]
            + ";; Genre: "
            + self.data.get_genres_by_id([final_items[i]])[0]
            for i in range(len(sorted_item_names))
        ]
        return eb_item

    def get_full_rankings(self, use_test=False, batch_size=512):
        """
        Compute full rankings for all users in self.data.
        - training items are pushed to the end
        - optionally, ground-truth test items can be put on top
        """
        if self.config['rec_model'] == 'MF':
           n_users = self.data.get_user_num()
           n_items = self.data.get_item_num()

           item_embed = self.model.item_embedding.weight[:n_items, :]

           self.full_rankings = np.zeros((n_users, n_items), dtype=int)

           for user in range(n_users):
              user_tensor = torch.tensor([user])
              user_embed = self.model.user_embedding(user_tensor)

              scores = torch.matmul(user_embed, item_embed.T).squeeze(0).detach().numpy()

              # Only consider valid item indices
              train_items = [x[1] for x in self.train_data if x[0] == user and x[1] < n_items]
              scores[train_items] = -np.inf

              self.full_rankings[user] = np.argsort(-scores)

              # # Optionally move ground-truth test items on top
              if use_test:
                 test_items = [x[1] for x in self.test_data if x[0] == user and x[1] < n_items]
                 for idx, item in enumerate(test_items):
                    if item in self.full_rankings[user]:
                       current_pos = np.where(self.full_rankings[user] == item)[0][0]
                       self.full_rankings[user][idx], self.full_rankings[user][current_pos] = (
                         self.full_rankings[user][current_pos],
                         self.full_rankings[user][idx])

        elif self.config['rec_model'] == 'LightGCN':
            n_users = self.data.get_user_num()
            n_items = self.data.get_item_num()

            # === 1. Get all user/item embeddings from LightGCN ===
            self.model.eval()
            with torch.no_grad():
                all_user_emb, all_item_emb = self.model.propagate()
                # shapes: (n_users, embed_dim), (n_items, embed_dim)

            self.full_rankings = np.zeros((n_users, n_items), dtype=int)

            for user in range(n_users):
               # Get user embedding
               user_embed = all_user_emb[user].unsqueeze(0)   # (1, embed_dim)

               # Compute scores for all items (dot product)
               scores = torch.matmul(user_embed, all_item_emb.T).squeeze(0).cpu().numpy()

               # Push training items to -inf
               train_items = [x[1] for x in self.train_data if x[0] == user and x[1] < n_items]
               scores[train_items] = -np.inf

               # Sort descending
               self.full_rankings[user] = np.argsort(-scores)

               # # Optionally move ground-truth test items on top
               if use_test:
                  test_items = [x[1] for x in self.test_data if x[0] == user and x[1] < n_items]
                  for idx, item in enumerate(test_items):
                     if item in self.full_rankings[user]:
                        current_pos = np.where(self.full_rankings[user] == item)[0][0]
                        self.full_rankings[user][idx], self.full_rankings[user][current_pos] = (
                           self.full_rankings[user][current_pos],
                           self.full_rankings[user][idx])

        elif self.config['rec_model'] == 'MultiVAE':
            # n_users = self.data.get_user_num()
            # n_items = self.data.get_item_num()
            if use_test:
               matrix = self.test_matrix
            else:
               matrix = self.train_matrix

            n_users, n_items = matrix.shape

            self.model.eval()
            self.full_rankings = np.zeros((n_users, n_items), dtype=int)

            with torch.no_grad():
                for start in range(0, n_users, batch_size):
                   end = min(start + batch_size, n_users)

                   # === 1. Build user input batch (interaction vectors) ===
                   batch_users = []
                   for u in range(start, end):
                      row = self.train_matrix[u]  # should return (n_items,) vector
                      batch_users.append(row)
                   batch_users = torch.tensor(batch_users, dtype=torch.float32).to(self.model.device)

                   # === 2. Forward pass through MultiVAE ===
                   logits, mu, logvar = self.model(batch_users)   # shape: (batch_size, n_items)
                   scores = logits.cpu().numpy()

                   # === 3. Postprocess each user in batch ===
                   for i, u in enumerate(range(start, end)):
                      user_scores = scores[i]

                      train_items = [x[1] for x in self.train_data if x[0] == u and x[1] < n_items]
                      user_scores[train_items] = -np.inf

                      # Sort items by descending score
                      self.full_rankings[u] = np.argsort(-user_scores)

                      # Optionally move ground-truth test items on top
                      if use_test:
                         test_items = [x[1] for x in self.test_data if x[0] == u and x[1] < n_items]
                         for idx, item in enumerate(test_items):
                            if item in self.full_rankings[u]:
                               current_pos = np.where(self.full_rankings[u] == item)[0][0]
                               self.full_rankings[u][idx], self.full_rankings[u][current_pos] = (
                                   self.full_rankings[u][current_pos],
                                   self.full_rankings[u][idx])

        elif self.config['rec_model'] == 'FM':
            n_users = self.data.get_user_num()
            n_items = self.data.get_item_num()

            self.full_rankings = np.zeros((n_users, n_items), dtype=int)

            self.model.eval()
            device = next(self.model.parameters()).device

            for user in range(n_users):
               # Generate all item IDs
               item_ids = torch.arange(n_items, device=device)
               user_ids = torch.full((n_items,), user, dtype=torch.long, device=device)

               # Compute scores using FM forward
               with torch.no_grad():
                  scores = self.model(user_ids, item_ids).cpu().numpy()

               # Push training items to the end
               train_items = [x[1] for x in self.train_data if x[0] == user and x[1] < n_items]
               scores[train_items] = -np.inf

               # Sort items by descending score
               ranking = np.argsort(-scores)

               # Optionally move ground-truth test items to the top
               if use_test:
                  test_items = [x[1] for x in self.test_data if x[0] == user and x[1] < n_items]
                  for idx, item in enumerate(test_items):
                     if item in ranking:
                        current_pos = np.where(ranking == item)[0][0]
                        ranking[idx], ranking[current_pos] = ranking[current_pos], ranking[idx]

               self.full_rankings[user] = ranking

        elif self.config['rec_model'] == 'SASRec':
            n_users = self.data.get_user_num()
            n_items = self.data.get_item_num()

            self.full_rankings = np.zeros((n_users, n_items), dtype=int)
            self.model.eval()
            device = next(self.model.parameters()).device

            with torch.no_grad():
               for start in range(0, n_users, batch_size):
                  end = min(start + batch_size, n_users)
                  batch_users = list(range(start, end))

                  # Build input sequences for batch
                  batch_seqs = []
                  for u in batch_users:
                     # Get user interaction sequence from train_data
                     user_items = [x[1] for x in self.train_data if x[0] == u]
                     padded_seq = _pad_sequence(user_items, self.model.max_seq_len)
                     batch_seqs.append(padded_seq)

                  batch_seqs = torch.tensor(batch_seqs, dtype=torch.long, device=device)

                  # Forward pass: get sequence embeddings
                  seq_out = self.model(batch_seqs)  # (B, L, H)
                  seq_out_last = seq_out[:, -1, :]  # use last position (B, H)

                  # All item embeddings
                  all_item_emb = self.model.item_embedding.weight[:n_items, :]  # (n_items, H)

                  # Compute scores
                  scores = torch.matmul(seq_out_last, all_item_emb.T)  # (B, n_items)
                  scores = scores.cpu().numpy()

                  # Mask training items
                  for i, u in enumerate(batch_users):
                     train_items = [x[1] for x in self.train_data if x[0] == u and x[1] < n_items]
                     scores[i, train_items] = -np.inf  # push train items to the end

                     ranking = np.argsort(-scores[i])  # full ranking by score (highest first)
                     if use_test:
                        test_items = [x[1] for x in self.test_data if x[0] == u and x[1] < n_items]
                        # Keep only test items that appear in ranking
                        test_items_in_ranking = [item for item in ranking if item in test_items]
                        # Take at most 5 test items
                        top_test_items = test_items_in_ranking[:5]
                        # Remaining items (exclude the ones we forced to the top)
                        other_items = [item for item in ranking if item not in top_test_items]

                        # New ranking: top test items first, then the rest in score order
                        ranking = np.array(top_test_items + other_items)
                     # Store final ranking
                     self.full_rankings[u] = ranking

In [ ]:
import random

class MovieLensDocument(AbstractDocument):
  def __init__(self, doc_id, genre_vector, avg_rating):
    """Initializes a MovieLens document with genre information and average rating."""
    self.genre_vector = genre_vector  # A vector encoding the movie's genres
    self.avg_rating = avg_rating  # Average rating of the movie
    super(MovieLensDocument, self).__init__(doc_id)

  def create_observation(self):
    """Returns observable properties of this document as a float array."""
    return np.concatenate([self.genre_vector, [self.avg_rating]])

  @staticmethod
  def observation_space():
    """Defines the observation space for MovieLens documents."""
    return spaces.Box(shape=(num_genres + 1,), dtype=np.float32, low=0.0, high=5.0)

  def __str__(self):
    return "Movie {} with genres {} and avg rating {:.2f}.".format(self._doc_id, self.genre_vector, self.avg_rating)


class MovieLensDocumentSampler(AbstractDocumentSampler):
    def __init__(self, recsys, total_user, page_size=5, users=None, doc_ctor=MovieLensDocument, **kwargs):
        """
        Initializes the sampler for MovieLens documents.
        Args:
            recsys: recommendation system object with full_rankings[user_id]
            total_user: number of users to sample from
            page_size: number of recommendations per page
            users: list of user IDs to iterate over (default = first N users)
        """
        super(MovieLensDocumentSampler, self).__init__(doc_ctor, **kwargs)
        self.recsys = recsys
        self.page_size = page_size
        self._doc_count = {}   # track per-user doc count
        self._page_index = {}  # track per-user page index

        if users is None:
            self.users = ratings_df['user_id'].unique()[:total_user]
        else:
            self.users = users

        # initialize tracking per user
        for uid in self.users:
            self._doc_count[uid] = 0
            self._page_index[uid] = 0

    def sample_document(self, user_id):
        """Return one document for the given user_id."""
        if user_id not in self.users:
            raise ValueError(f"User {user_id} not found in sampler users.")

        self._doc_count[user_id] += 1

        # Get this user's recommendation list
        recommended_items = self.recsys.full_rankings[user_id]
        recommended_items_list = recommended_items.tolist() if not isinstance(recommended_items, list) else recommended_items

        # Compute start & end indices for current page
        start_idx = self._page_index[user_id] * self.page_size
        end_idx = start_idx + self.page_size

        # Pick item within this page
        item_idx = start_idx + (self._doc_count[user_id] - 1) % self.page_size
        if item_idx >= len(recommended_items_list):
            # No more recommendations left → reset this user
            self._doc_count[user_id] = 0
            self._page_index[user_id] = 0
            return None

        item_id = recommended_items_list[item_idx]
        # print("user id: ", user_id, "item id: ", item_id)

        # Get movie info
        movie = movies_df[movies_df['item_id'] == item_id].iloc[0]

        # Encode genres
        genre_vector = np.zeros(num_genres)
        for genre in movie['processed_genre'].split('|'):
            if genre in genre_to_index:
                genre_vector[genre_to_index[genre]] = 1

        # Avg rating
        avg_rating = avg_ratings.get(movie['item_id'], 0.0)

        doc_features = {
            'doc_id': movie['item_id'],
            'genre_vector': genre_vector,
            'avg_rating': avg_rating
        }

        # When we finish a page (doc_count == page_size), move to next page
        if self._doc_count[user_id] % self.page_size == 0:
            self._page_index[user_id] += 1

        return self._doc_ctor(**doc_features)

In [ ]:
class MovieLensUserState(AbstractUserState):
    """Represents a user's preference state based on MovieLens data."""

    def __init__(self, genre_preferences, rating_sensitivity, observation_noise_stddev=0.1):
        """
        Initializes a user with genre preferences and rating sensitivity.

        Args:
            genre_preferences: A vector representing the user's preference for each genre.
            rating_sensitivity: A value that influences how sensitive the user is to rating differences.
            observation_noise_stddev: Standard deviation of observation noise.
        """
        self.genre_preferences = np.array(genre_preferences)
        self.rating_sensitivity = rating_sensitivity
        self._observation_noise = observation_noise_stddev
        self.satisfaction = 0.5  # Initial satisfaction level
        self.time_budget = 10  # Simulation steps user is active

    def create_observation(self):
        """Returns a noisy observation of user preferences."""
        noise = np.random.normal(0, self._observation_noise, len(self.genre_preferences))
        noisy_preferences = self.genre_preferences + noise
        return np.clip(noisy_preferences, 0, 1)  # Ensure values stay within [0,1]

    @staticmethod
    def observation_space():
        """Defines the observation space for user preferences."""
        return spaces.Box(shape=(num_genres,), dtype=np.float32, low=0.0, high=1.0)

    def score_document(self, doc_obs):
        """Computes a compatibility score between the user and the movie.

        Args:
            doc_obs: A movie's genre vector.

        Returns:
            A score indicating user interest in the movie.
        """
        genre_match = np.dot(self.genre_preferences, doc_obs[:-1])  # Ignore avg rating in doc_obs
        return genre_match * self.rating_sensitivity


class MovieLensUserSampler(AbstractUserSampler):
    """Samples users with different genre preferences based on the data."""

    def __init__(self, user_id, user_ctor=MovieLensUserState, user_genres=None, genre_to_index=None, seed=42):
        super().__init__(user_ctor, seed=seed)
        self.user_genres = user_genres
        self.genre_to_index = genre_to_index
        self.user_id = user_id  # Store user_id internally

    def sample_user(self):
        if self.user_id is None:
            raise ValueError("User ID has not been set. Use set_user_id() before sampling.")

        user_row = self.user_genres[self.user_genres['user_id'] == self.user_id]
        if user_row.empty:
            raise ValueError(f"User with user_id {self.user_id} not found in the dataset.")

        user_row = user_row.iloc[0]
        genre_str = user_row['processed_genre']
        genres = genre_str.split('|')
        genre_indices = [self.genre_to_index.get(genre, -1) for genre in genres if genre in self.genre_to_index]
        genre_preferences = np.zeros(len(self.genre_to_index))
        genre_preferences[genre_indices] = 1
        genre_preferences /= genre_preferences.sum()

        rating_sensitivity = np.random.uniform(0.5, 2.0)
        return self._user_ctor(genre_preferences, rating_sensitivity)


class MovieLensResponse(AbstractResponse):
    """Models user responses based on their preferences."""

    def __init__(self, clicked=False, engagement=0.0, rating=0.0):
        self.clicked = clicked
        self.engagement = engagement
        self.rating = rating

    def create_observation(self):
        """Creates a response observation."""
        return {'click': int(self.clicked), 'engagement': self.engagement, 'rating': self.rating}

    @classmethod
    def response_space(cls):
        """Defines response space (click, engagement, rating)."""
        return spaces.Dict({
            'click': spaces.Discrete(2),
            'engagement': spaces.Box(low=0.0, high=100.0, shape=(), dtype=np.float32),
            'rating': spaces.Box(low=0.0, high=5.0, shape=(), dtype=np.float32)
        })



In [ ]:
class CustomChoiceModel(NormalizableChoiceModel):
    """Custom choice model to represent user behavior in a movie recommendation system."""

    def __init__(self, choice_features=None):
        if choice_features is None:
            choice_features = {}

        self._min_score = choice_features.get('min_score', 0.0)
        self._no_click_mass = choice_features.get('no_click_mass', -2.0)
        self._scores = None
        self._score_no_click = None

    def score_documents(self, user_state, doc_obs_list):
        """Computes scores for a list of documents based on user preferences."""
        scores = np.array([
            np.dot(user_state.genre_preferences, doc_obs[:-1]) + 0.3 * user_state.rating_sensitivity
            for doc_obs in doc_obs_list
        ])

        scores = np.append(scores, self._no_click_mass)  # Add a score for no-click option
        all_scores = self._softmax(scores)
        self._scores = all_scores[:-1]
        self._score_no_click = all_scores[-1]

    def choose_item(self, user_state, doc_obs_list, similarity_threshold=0.1):

        """
        Selects multiple documents based on genre similarity.

        Args:
        user_state: The user's current state, containing `genre_preferences`.
        doc_obs_list: List of document observations, each containing `genre_vector`.
        similarity_threshold (float, optional): Minimum similarity required to select an item.

        Returns:
        List of selected item indices (or empty list if none are selected).
        """

        # Compute genre similarity scores
        similarity_scores = np.array([
            np.dot(user_state.genre_preferences, doc_obs[:-1])  # Assuming last element isn't genre-related
            for doc_obs in doc_obs_list])

        # Normalize scores to get probabilities
        similarity_probs = similarity_scores - np.min(similarity_scores)  # Ensure non-negative


        if np.sum(similarity_probs) > 0:
            similarity_probs /= np.sum(similarity_probs)
        else:
            return []  # No valid selections

        # Select items that meet the similarity threshold
        selected_indices = [i for i, score in enumerate(similarity_probs) if score >= similarity_threshold]

        return selected_indices  # Returns indices of selected movies

    def _softmax(self, vector):
        """Numerically stable softmax function."""
        vector = np.array(vector)
        exp_vector = np.exp(vector - np.max(vector))  # Subtract max to prevent overflow
        return exp_vector / np.sum(exp_vector)



In [ ]:
class MovieLensUserModel(AbstractUserModel):
    """MovieLens-1M User Model, integrating preferences and choice models."""

    def __init__(self, user_id, slate_size, seed=0):
        super(MovieLensUserModel, self).__init__(
            MovieLensResponse,
            MovieLensUserSampler(user_id= user_id, user_genres=user_genres, genre_to_index=genre_to_index, seed=int(seed)),
            slate_size
        )
        np.random.seed(seed)
        self.user_id = user_id
        self.users_df = users_df
        self.user_genres = user_genres
        self.slate_size = slate_size
        self.choice_model = CustomChoiceModel({'min_score': 0.0, 'no_click_mass': -2.0})
        self._initialize_user_state()

    def _initialize_user_state(self):
        """Initializes a random user from the dataset with preferences."""
        # self.user_data = self.data[self.data["user_id"] == self.user_id]
        # genre_preferences = np.random.rand(num_genres)  # Random preference for each genre

        self.user_data = self.user_genres[self.user_genres["user_id"] == self.user_id]
        if self.user_data.empty:
           raise ValueError(f"User {self.user_id} not found in user_genres")

        # Extract genre preferences
        user_row = self.user_data.iloc[0]
        genre_str = user_row["processed_genre"]
        genres = genre_str.split('|')
        genre_indices = [genre_to_index[g] for g in genres if g in genre_to_index]

        genre_preferences = np.zeros(len(genre_to_index))
        if genre_indices:
           genre_preferences[genre_indices] = 1
           genre_preferences /= genre_preferences.sum()
        else:
           # fallback to random vector if no valid genres found
           genre_preferences = np.random.rand(num_genres)
           genre_preferences /= genre_preferences.sum()

        self._user_state = MovieLensUserState(
            genre_preferences, rating_sensitivity=np.random.uniform(0.5, 2.0)
        )
        self._user_state.satisfaction = 0.5  # Initial satisfaction level
        self._user_state.time_budget = 20  # Simulation steps user is active

    def simulate_response(self, slate_movies):
        """Simulates user responses to a set of recommended movies."""
        responses = [MovieLensResponse(clicked=False, engagement=0.0, rating=0.0) for _ in slate_movies]

        # Convert slate movies to document observations
        movie_observations = [movie.create_observation() for movie in slate_movies]

        # Score documents and select one
        self.choice_model.score_documents(self._user_state, movie_observations)
        selected_indices = self.choice_model.choose_item(self._user_state, movie_observations)

        for selected_index in selected_indices:
            self._generate_response(slate_movies[selected_index], responses[selected_index])

        return responses


    def _generate_response(self, movie, response):
        """Generates user engagement response for a selected movie."""
        response.clicked = True
        engagement_loc = np.dot(movie.genre_vector, self._user_state.genre_preferences)
        log_engagement = np.random.normal(loc=engagement_loc, scale=0.5)
        response.engagement = np.exp(log_engagement)
        # response.rating = (movie.avg_rating)*2 + np.random.normal(0, 0.5)  # Small noise in rating
        # Adjust the rating generation using user preferences and sensitivity
        base_rating = movie.avg_rating  # The movie's average rating

        # Apply genre preference and rating sensitivity to modify the base rating
        genre_influence = np.dot(movie.genre_vector, self._user_state.genre_preferences)
        sensitivity_factor = self._user_state.rating_sensitivity

        # Use a weighted sum to determine the final rating with some noise
        rating = base_rating + genre_influence * sensitivity_factor + np.random.normal(0, 0.5)

        # Ensure the rating is within a reasonable range (0 to 5 for example)
        response.rating = np.clip(rating, 0, 5)

    def update_state(self, slate_movies, responses):
        """Updates user state based on interaction with the recommended slate."""
        for movie, response in zip(slate_movies, responses):
            if response.clicked:
                innovation = np.random.normal(scale=0.1)
                self._user_state.satisfaction *= 0.9  # Memory decay for satisfaction
                self._user_state.satisfaction += innovation
                self._user_state.time_budget -= 1  # Reduce remaining time

    def is_terminal(self):
        """Checks if the user has exhausted their time budget."""
        return self._user_state.time_budget <= 0


In [ ]:
class MovieLensMultiUserEnvironment(MultiUserEnvironment):
    """Multi-user environment for MovieLens user models, each with its own candidate set."""

    def __init__(self, user_models, document_sampler, num_candidates, slate_size, resample_documents=False):
        """
        Args:
            user_models (list): List of MovieLensUserModel instances.
            document_sampler: AbstractDocumentSampler for movies.
            num_candidates (int): Number of candidate docs per user.
            slate_size (int): Slate size for recommendations.
            resample_documents (bool): If True, resample docs each step.
        """
        self._user_models = user_models
        self._document_sampler = document_sampler
        self._num_candidates = num_candidates
        self._slate_size = slate_size
        self._resample_documents = resample_documents

        # Each user gets their own candidate set
        self._candidate_sets = {user_model.user_id: CandidateSet() for user_model in user_models}
        self._current_documents = {}

    @property
    def user_model(self):
        return self._user_models

    @property
    def num_users(self):
        return len(self._user_models)

    def _do_resample_documents(self):
        """Resample candidate sets for all users."""
        for user_model in self._user_models:
            candidate_set = CandidateSet()
            docs = [
                self._document_sampler.sample_document(user_id=user_model.user_id)
                for _ in range(self._num_candidates)
            ]
            for d in docs:
                if d is not None:
                    candidate_set.add_document(d)
            self._candidate_sets[user_model.user_id] = candidate_set

    def reset(self):
        """Reset all users and their candidate sets, return initial obs."""
        for user_model in self._user_models:
            user_model.reset()

        # Build candidate sets
        self._do_resample_documents()

        user_obs = [user_model.create_observation() for user_model in self._user_models]

        # Create document observations per user
        self._current_documents = {
            user_model.user_id: collections.OrderedDict(
                self._candidate_sets[user_model.user_id].create_observation()
            )
            for user_model in self._user_models
        }

        return user_obs, self._current_documents

    def step(self, slates):
        """Run one simulation step for all users.

        Args:
             slates (list of list): Each element is a slate (list of indices) for that user.
        """
        assert len(slates) == self.num_users, (
            f"Expected {self.num_users} slates, got {len(slates)}"
            )

        user_obs_list = []
        current_documents_list = []
        responses_list = []

        for user_model, slate in zip(self._user_models, slates):
            if isinstance(slate, (int, np.integer)):
               slate = [slate]   # wrap single int into a list

            doc_ids = list(self._current_documents[user_model.user_id].keys())
            mapped_slate = [doc_ids[x] for x in slate]
            documents = self._candidate_sets[user_model.user_id].get_documents(mapped_slate)
            if user_model.is_terminal():
               responses = [MovieLensResponse(clicked=False, engagement=0.0, rating=0.0) for _ in documents]
            else:
               responses = user_model.simulate_response(documents)
               user_model.update_state(documents, responses)

            user_obs_list.append(user_model.create_observation())
            current_documents_list.append(documents)
            responses_list.append(responses)

        # Update sampler with all docs & responses (flattened)
        def flatten(lst):
            return list(itertools.chain(*lst))

        self._document_sampler.update_state(
            flatten(current_documents_list),
            flatten(responses_list)
        )

        done = all(user_model.is_terminal() for user_model in self._user_models)

        if self._resample_documents:
           self._do_resample_documents()

        # Refresh current docs per user
        self._current_documents = {
        user_model.user_id: collections.OrderedDict(
            self._candidate_sets[user_model.user_id].create_observation()
            )
        for user_model in self._user_models
        }
        return user_obs_list, current_documents_list, responses_list, done

        # return user_obs_list, self._current_documents, responses_list, done

In [ ]:
# Assuming RecSimGymEnv, MovieLensSingleUserEnvironment, and other relevant classes are already imported

def total_clicks_reward(responses_list):
    """Compute total clicks across all users and all movies in the slate."""
    reward = 0
    for user_responses in responses_list:          # iterate per-user
        for r in user_responses:                  # iterate per-movie
            reward += r.clicked
    return reward

def aggregate_video_cluster_metrics(responses, metrics, info=None):
    """Aggregates the video cluster metrics with one step responses."""
    del info  # Unused.
    is_clicked = False
    metrics['impression'] += 1

    for response in responses:
        if not response['click']:
            continue
        is_clicked = True
        metrics['click'] += 1
        metrics['quality'] += response['quality']
        cluster_id = response['cluster_id']
        metrics['cluster_watch_count_cluster_%d' % cluster_id] += 1

    if not is_clicked:
        metrics['cluster_watch_count_no_click'] += 1
    return metrics

def write_video_cluster_metrics(metrics, add_summary_fn):
    """Writes average video cluster metrics using add_summary_fn."""
    add_summary_fn('CTR', metrics['click'] / metrics['impression'])
    if metrics['click'] > 0:
        add_summary_fn('AverageQuality', metrics['quality'] / metrics['click'])
    for k in metrics:
        prefix = 'cluster_watch_count_cluster_'
        if k.startswith(prefix):
            add_summary_fn('cluster_watch_count_frac/cluster_%s' % k[len(prefix):],
                           metrics[k] / metrics['impression'])
    add_summary_fn(
        'cluster_watch_count_frac/no_click',
        metrics['cluster_watch_count_no_click'] / metrics['impression'])

In [ ]:
from collections import defaultdict


class EpsilonGreedyRecommenderAgent(AbstractEpisodicRecommenderAgent):
    """A simple ε-greedy recommender agent for MovieLens recommendations."""

    def __init__(self, action_space, slate_size, epsilon=0.1, summary_writer=None):
        """
        Initializes the recommender agent.

        Args:
            action_space: The available action space.
            slate_size: Number of movies to recommend in each step.
            epsilon: Probability of exploration.
            summary_writer: For logging metrics (if applicable).
        """
        super().__init__(action_space, summary_writer)
        self.epsilon = epsilon
        self.slate_size = slate_size
        self.q_values = defaultdict(float)  # Stores Q-values for movies
        self.action_counts = defaultdict(int)  # Tracks number of times each movie is recommended
        self.last_action = None  # Stores last recommended slate
        self.last_docs = None          # new: list of doc_ids corresponding to last_action

        # New: track sequential selection
        self._next_start_idx = 0


    def step(self, reward, observation):
        # Update Q-values first (if reward is provided)
        if reward is not None and self.last_action is not None and self.last_docs is not None:
            self._update_q_values(reward, self.last_action, self.last_docs)

        doc_ids = list(observation["doc"].keys())
        num_candidates = len(doc_ids)

        # Sequential selection of slate
        start_idx = self._next_start_idx
        end_idx = start_idx + self.slate_size
        slate_indices = list(range(start_idx, min(end_idx, num_candidates)))

        # Wrap around if exceeding number of candidates
        if end_idx > num_candidates:
            overflow = end_idx - num_candidates
            slate_indices += list(range(0, overflow))

        # Update _next_start_idx for next step
        self._next_start_idx = (self._next_start_idx + self.slate_size) % num_candidates

        # Store last action and mapping
        self.last_action = slate_indices
        self.last_docs = doc_ids

        return slate_indices

    def _update_q_values(self, reward_dict, action, last_docs):
        """
        reward_dict: dict mapping doc_id -> reward
        action: list of indices (slate)
        last_docs: list mapping indices -> doc_id
        """
        for idx in action:
            doc_id = last_docs[idx]
            r = reward_dict.get(doc_id, 0.0)
            self.action_counts[doc_id] += 1
            self.q_values[doc_id] += (r - self.q_values[doc_id]) / self.action_counts[doc_id]


    def _select_best_slate(self, doc_obs_list):
        """
        Selects movies with the highest estimated Q-values.

        Args:
            doc_obs_list: List of available document observations (movies).

        Returns:
            A list of selected movie indices.
        """
        movie_ids = list(range(len(doc_obs_list)))

        if len(self.q_values) < self.slate_size:
            # If we haven't explored enough, pick randomly
            return np.random.choice(movie_ids, self.slate_size, replace=False).tolist()

        # Sort movie IDs by Q-value in descending order and pick the top ones
        sorted_movies = sorted(movie_ids, key=lambda movie: self.q_values[movie], reverse=True)
        return sorted_movies[:self.slate_size]

    def bundle_and_checkpoint(self, checkpoint_dir, iteration_number):
        """
        Saves the agent's internal state for checkpointing.

        Args:
            checkpoint_dir: Directory to store checkpoint data.
            iteration_number: Current training iteration.

        Returns:
            Dictionary containing saved Q-values and action counts.
        """
        return {
            "q_values": dict(self.q_values),
            "action_counts": dict(self.action_counts)
        }

    def unbundle(self, checkpoint_dir, iteration_number, bundle_dict):
        """
        Restores the agent's state from a checkpoint.

        Args:
            checkpoint_dir: Directory containing checkpoint data.
            iteration_number: The iteration at which the checkpoint was saved.
            bundle_dict: Dictionary containing saved state.

        Returns:
            Boolean indicating whether restoration was successful.
        """
        if "q_values" in bundle_dict:
            self.q_values = defaultdict(float, bundle_dict["q_values"])
        if "action_counts" in bundle_dict:
            self.action_counts = defaultdict(int, bundle_dict["action_counts"])
        return True
